In [ ]:
import json
import sys
import geopandas as gpd
import matplotlib as mpl
import matplotlib.colors as colors
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import numpy as np
import osmnx as ox
import pandas as pd
import scipy.integrate as integrate
from ast import literal_eval
from os.path import join
from pathlib import Path
from scipy import stats
from setup_paths import paths
sys.path.append(paths["project_dir"])

In [ ]:
def million_to_billion(df):
    df["CC_t"] = df["CC_t"] / 1000
    df["SV_t"] = df["SV_t"] / 1000
    df["MC_t"] = df["MC_t"] / 1000
    df["HB_t"] = df["HB_t"] / 1000
    df["NPV_t"] = df["NPV_t"] / 1000
    df["CumulCC_t"] = df["CumulCC_t"] / 1000
    df["CumulSV_t"] = df["CumulSV_t"] / 1000
    df["CumulMC_t"] = df["CumulMC_t"] / 1000
    df["CumulTB_t"] = df["CumulTB_t"] / 1000
    df["CumulHB_t"] = df["CumulHB_t"] / 1000
    df["CurrentNPV"] = df["CurrentNPV"] / 1000
    return df

In [ ]:
def load_yearly_ordering(tud, dtu):
    with open(join(paths["output_folder"], tud, f"{tud}_order_yearly_Penalty.json"), "r") as f:
      tud_yearly_p = json.load(f)
    tud_yearly_p = {int(k[4:]): v for k, v in tud_yearly_p.items()}
    tud_yearly_p[0] = []
    with open(join(paths["output_folder"], tud, f"{tud}_order_yearly_TB.json"), "r") as f:
      tud_yearly_tb = json.load(f)
    tud_yearly_tb = {int(k[4:]): v for k, v in tud_yearly_tb.items()}
    tud_yearly_tb[0] = []
    with open(join(paths["output_folder"], tud, f"{tud}_order_yearly_TBHB.json"), "r") as f:
      tud_yearly_tbhb = json.load(f)
    tud_yearly_tbhb = {int(k[4:]): v for k, v in tud_yearly_tbhb.items()}
    tud_yearly_tbhb[0] = []
    
    with open(join(paths["output_folder"], dtu, f"{dtu}_order_yearly_LPGreedy.json"), "r") as f:
      dtu_yearly_greedy = json.load(f)
    dtu_yearly_greedy = {int(k[4:]): v for k, v in dtu_yearly_greedy.items()}
    dtu_yearly_greedy[0] = []
    with open(join(paths["output_folder"], dtu, f"{dtu}_order_yearly_TA.json"), "r") as f:
      dtu_yearly_ta = json.load(f)
    dtu_yearly_ta = {int(k[4:]): v for k, v in dtu_yearly_ta.items()}
    dtu_yearly_ta[0] = []
    
    return tud_yearly_p, tud_yearly_tb, tud_yearly_tbhb, dtu_yearly_greedy, dtu_yearly_ta

In [ ]:
def get_yearly_added(yearly):
    yearly_added = {year: set(yearly[year])-set(yearly[year-1]) for year in range(1,51)}
    segment_year = {segment: year for year, year_segments in yearly_added.items() for segment in year_segments}
    
    return yearly_added, segment_year

In [ ]:
def total_deviation(noisy_data, base_data, time):
    diff_data = pd.DataFrame({"t": time})
    diff_data["diff"] = noisy_data-base_data
    
    return abs(integrate.simpson(diff_data[diff_data["diff"]>0]["diff"], x=diff_data[diff_data["diff"]>0]["t"])) + abs(integrate.simpson(diff_data[diff_data["diff"]<0]["diff"], x=diff_data[diff_data["diff"]<0]["t"]))

In [ ]:
mpl.rcParams.update(mpl.rcParamsDefault)
mpl.rcParams["mathtext.fontset"] = "custom"
mpl.rcParams["mathtext.it"] = "Arial:italic"
mpl.rcParams["mathtext.rm"] = "Arial"

In [ ]:
tud_save = "cph_tud"
dtu_save = "cph_dtu"

In [ ]:
tud_comp = "tud_tbhb"
dtu_comp = "dtu_ta"

In [ ]:
figures = join(paths["plot_folder"], "results", "cph")
Path(figures).mkdir(parents=True, exist_ok=True)
currency_conv = 7.44963
last_year = 50
segments_added = range(0, 203)
all_years = range(0, 51)
years_to_plot = range(0, last_year+1)
noise_samples = 10
outlier_threshold = 50

In [ ]:
figsize = (2.4, 2.0)
dpi = 300
axis_fs = 8
tick_fs = 6

In [ ]:
file_format = "svg"

In [ ]:
color_dtu_greedy = "#ffc819"
color_dtu_ta = "#f58231"
color_tud_p = "#000075"
color_tud_tb = "#4363d8"
color_tud_tbhb = "#42d4f4"
color_combined = "#7b4254"    #"#9cac93"

color_street_nw = "#cdcdcd"
color_exising_sch = "#999999"
color_upgradable_sch = "#ebe1d7"

color = {"color_dtu_greedy": color_dtu_greedy, "color_dtu_ta": color_dtu_ta,"color_tud_p": color_tud_p, "color_tud_tb": color_tud_tb, "color_tud_tbhb": color_tud_tbhb, "color_both": color_combined, "color_street_nw": color_street_nw, "color_exising_sch": color_exising_sch, "color_upgradable_sch": color_upgradable_sch}

In [ ]:
edge_dtypes = {
    "NormalInfraType": int,
    "UpgradedInfraType": int,
    "IntersectionDelay": int,
    "seg_id": int,
    "cost": float,
}
g = ox.load_graphml(join(paths["input_folder"], tud_save, f"{tud_save}.graphml"), edge_dtypes=edge_dtypes)
for u, v, d in g.edges(data=True):
    d["Active_Basis"] = literal_eval(d["Active_Basis"])
    d["ex_inf"] = literal_eval(d["ex_inf"])
    d["blocked"] = literal_eval(d["blocked"])
    d["bike_highway"] = literal_eval(d["bike_highway"])

In [ ]:
segment_cost = pd.read_csv(join(paths["input_folder"], f"{tud_save}", "SegmentCosts.csv"))
segment_cost.drop(segment_cost[segment_cost.Length == 0].index, inplace=True)
segment_cost.drop(["RouteId"], axis=1, inplace=True)
segment_cost["ConstructionCosts"] = segment_cost["ConstructionCosts"] / currency_conv
segment_cost["MaintenanceCosts"] = segment_cost["MaintenanceCosts"] / currency_conv
total_segment_cost = segment_cost["ConstructionCosts"].sum()

In [ ]:
segment_length = {i: 0 for i in range(1, 208)}
for u, v, d in g.edges(data=True):
    if d["seg_id"] != -1:
        segment_length[d["seg_id"]] += d["length"] / 1000
segment_cost["LengthGraph"] = [segment_length[row["SegmentId"]] for index, row in segment_cost.iterrows()]
segment_cost["CC per km"] = segment_cost["ConstructionCosts"] / segment_cost["LengthGraph"]
segment_cost["MC per km"] = segment_cost["MaintenanceCosts"] / segment_cost["LengthGraph"]
total_segment_length = segment_cost["LengthGraph"].sum()

In [ ]:
geom_streetnw = gpd.read_file(join(paths["data_dir"], "raw", "CPH", "CPH_StreetNW_Geometry.gpkg"))
geom_streetnw.drop_duplicates(inplace=True)
geom_existing = gpd.read_file(join(paths["data_dir"], "raw", "CPH", "CPH_ExistingSCH_Geometry.gpkg"))
geom_existing.drop_duplicates(inplace=True)
geom_proposed = gpd.read_file(join(paths["data_dir"], "raw", "CPH", "CPH_ProposedSCH_Geometry.gpkg"))
geom_proposed.drop_duplicates()
geom_demand = gpd.read_file(join(paths["data_dir"], "raw", "CPH", "CPH_Demand_Geometry.gpkg"))

In [ ]:
with open(join(paths["output_folder"], tud_save, f"{tud_save}_utility_Penalty.json"), "r") as f:
  tud_p_utility_data = json.load(f)
with open(join(paths["output_folder"], tud_save, f"{tud_save}_utility_TB.json"), "r") as f:
  tud_tb_utility_data = json.load(f)
with open(join(paths["output_folder"], tud_save, f"{tud_save}_utility_TBHB.json"), "r") as f:
  tud_tbhb_utility_data = json.load(f)

with open(join(paths["output_folder"], dtu_save, f"{dtu_save}_utility_LPGreedy_eval.json"), "r") as f:
  dtu_greedy_utility_data = json.load(f)
with open(join(paths["output_folder"], dtu_save, f"{dtu_save}_utility_TA_eval.json"), "r") as f:
  dtu_ta_utility_data = json.load(f)

In [ ]:
monetary_data = pd.DataFrame({"t": range(1, 51)})
tud_p_monetary_data = million_to_billion(pd.read_csv(join(paths["output_folder"], f"{tud_save}", "Evaluation_EUROs_TUDPenalty_T50.csv")))
monetary_data["tud_p_tb"] = tud_p_monetary_data["CumulTB_t"]
monetary_data["tud_p_npv"] = tud_p_monetary_data["CurrentNPV"]
monetary_data["tud_p_hb"] = tud_p_monetary_data["CumulHB_t"]
tud_tb_monetary_data = million_to_billion(pd.read_csv(join(paths["output_folder"], f"{tud_save}", "Evaluation_EUROs_TUDTB_T50.csv")))
monetary_data["tud_tb_tb"] = tud_tb_monetary_data["CumulTB_t"]
monetary_data["tud_tb_npv"] = tud_tb_monetary_data["CurrentNPV"]
monetary_data["tud_tb_hb"] = tud_tb_monetary_data["CumulHB_t"]
tud_tbhb_monetary_data = million_to_billion(pd.read_csv(join(paths["output_folder"], f"{tud_save}", "Evaluation_EUROs_TUDTBHB_T50.csv")))
monetary_data["tud_tbhb_tb"] = tud_tbhb_monetary_data["CumulTB_t"]
monetary_data["tud_tbhb_npv"] = tud_tbhb_monetary_data["CurrentNPV"]
monetary_data["tud_tbhb_hb"] = tud_tbhb_monetary_data["CumulHB_t"]

dtu_greedy_monetary_data = million_to_billion(pd.read_csv(join(paths["output_folder"], f"{dtu_save}", "Evaluation_EUROs_LPGreedy_T50.csv")))
monetary_data["dtu_greedy_tb"] = dtu_greedy_monetary_data["CumulTB_t"]
monetary_data["dtu_greedy_npv"] = dtu_greedy_monetary_data["CurrentNPV"]
monetary_data["dtu_greedy_hb"] = dtu_greedy_monetary_data["CumulHB_t"]
dtu_ta_monetary_data = million_to_billion(pd.read_csv(join(paths["output_folder"], f"{dtu_save}", "Evaluation_EUROs_Nov26TA_T50.csv")))
monetary_data["dtu_ta_tb"] = dtu_ta_monetary_data["CumulTB_t"]
monetary_data["dtu_ta_npv"] = dtu_ta_monetary_data["CurrentNPV"]
monetary_data["dtu_ta_hb"] = dtu_ta_monetary_data["CumulHB_t"]

In [ ]:
monetary_data_noisy = pd.DataFrame({"t": range(1, 51)})
for noisy in ["noisy_costs", "noisy_demand", "noisy_speeds"]:
    for i in range(1, noise_samples+1):
        tud_save_noisy = f"{tud_save}_{noisy}_{i}"
        dtu_save_noisy = f"{dtu_save}_{noisy}_{i}"

        tud_p_monetary_data_noisy = million_to_billion(pd.read_csv(join(paths["output_folder"], tud_save_noisy, f"Evaluation_EUROs_TUDPenalty_{noisy}_{i}_T50.csv")))
        monetary_data_noisy[f"tud_p_{noisy}_{i}_tb"] = tud_p_monetary_data_noisy["CumulTB_t"]
        monetary_data_noisy[f"tud_p_{noisy}_{i}_hb"] = tud_p_monetary_data_noisy["CumulHB_t"]
        monetary_data_noisy[f"tud_p_{noisy}_{i}_npv"] = tud_p_monetary_data_noisy["CurrentNPV"]
        tud_tb_monetary_data_noisy = million_to_billion(pd.read_csv(join(paths["output_folder"], tud_save_noisy, f"Evaluation_EUROs_TUDTB_{noisy}_{i}_T50.csv")))
        monetary_data_noisy[f"tud_tb_{noisy}_{i}_tb"] = tud_tb_monetary_data_noisy["CumulTB_t"]
        monetary_data_noisy[f"tud_tb_{noisy}_{i}_hb"] = tud_tb_monetary_data_noisy["CumulHB_t"]
        monetary_data_noisy[f"tud_tb_{noisy}_{i}_npv"] = tud_tb_monetary_data_noisy["CurrentNPV"]
        tud_tbhb_monetary_data_noisy = million_to_billion(pd.read_csv(join(paths["output_folder"], tud_save_noisy, f"Evaluation_EUROs_TUDTBHB_{noisy}_{i}_T50.csv")))
        monetary_data_noisy[f"tud_tbhb_{noisy}_{i}_tb"] = tud_tbhb_monetary_data_noisy["CumulTB_t"]
        monetary_data_noisy[f"tud_tbhb_{noisy}_{i}_hb"] = tud_tbhb_monetary_data_noisy["CumulHB_t"]
        monetary_data_noisy[f"tud_tbhb_{noisy}_{i}_npv"] = tud_tbhb_monetary_data_noisy["CurrentNPV"]
        
        dtu_greedy_monetary_data_noisy = million_to_billion(pd.read_csv(join(paths["output_folder"], dtu_save_noisy, f"Evaluation_EUROs_LPGreedy_{noisy}_{i}_T50.csv")))
        monetary_data_noisy[f"dtu_greedy_{noisy}_{i}_tb"] = dtu_greedy_monetary_data_noisy["CumulTB_t"]
        monetary_data_noisy[f"dtu_greedy_{noisy}_{i}_hb"] = dtu_greedy_monetary_data_noisy["CumulHB_t"]
        monetary_data_noisy[f"dtu_greedy_{noisy}_{i}_npv"] = dtu_greedy_monetary_data_noisy["CurrentNPV"]
        dtu_ta_monetary_data_noisy = million_to_billion(pd.read_csv(join(paths["output_folder"], dtu_save_noisy, f"Evaluation_EUROs_Nov26TA_{noisy}_{i}_T50.csv")))
        monetary_data_noisy[f"dtu_ta_{noisy}_{i}_tb"] = dtu_ta_monetary_data_noisy["CumulTB_t"]
        monetary_data_noisy[f"dtu_ta_{noisy}_{i}_hb"] = dtu_ta_monetary_data_noisy["CumulHB_t"]
        monetary_data_noisy[f"dtu_ta_{noisy}_{i}_npv"] = dtu_ta_monetary_data_noisy["CurrentNPV"]
    
    monetary_data_noisy[f"tud_p_{noisy}_tb"] = monetary_data_noisy[[f"tud_p_{noisy}_{i}_tb" for i in range(1, noise_samples+1)]].mean(axis=1)
    monetary_data_noisy[f"tud_p_{noisy}_hb"] = monetary_data_noisy[[f"tud_p_{noisy}_{i}_hb" for i in range(1, noise_samples+1)]].mean(axis=1)
    monetary_data_noisy[f"tud_p_{noisy}_npv"] = monetary_data_noisy[[f"tud_p_{noisy}_{i}_npv" for i in range(1, noise_samples+1)]].mean(axis=1)
    monetary_data_noisy[f"tud_tb_{noisy}_tb"] = monetary_data_noisy[[f"tud_tb_{noisy}_{i}_tb" for i in range(1, noise_samples+1)]].mean(axis=1)
    monetary_data_noisy[f"tud_tb_{noisy}_hb"] = monetary_data_noisy[[f"tud_tb_{noisy}_{i}_hb" for i in range(1, noise_samples+1)]].mean(axis=1)
    monetary_data_noisy[f"tud_tb_{noisy}_npv"] = monetary_data_noisy[[f"tud_tb_{noisy}_{i}_npv" for i in range(1, noise_samples+1)]].mean(axis=1)
    monetary_data_noisy[f"tud_tbhb_{noisy}_tb"] = monetary_data_noisy[[f"tud_tbhb_{noisy}_{i}_tb" for i in range(1, noise_samples+1)]].mean(axis=1)
    monetary_data_noisy[f"tud_tbhb_{noisy}_hb"] = monetary_data_noisy[[f"tud_tbhb_{noisy}_{i}_hb" for i in range(1, noise_samples+1)]].mean(axis=1)
    monetary_data_noisy[f"tud_tbhb_{noisy}_npv"] = monetary_data_noisy[[f"tud_tbhb_{noisy}_{i}_npv" for i in range(1, noise_samples+1)]].mean(axis=1)

    monetary_data_noisy[f"dtu_greedy_{noisy}_tb"] = monetary_data_noisy[[f"dtu_greedy_{noisy}_{i}_tb" for i in range(1, noise_samples+1)]].mean(axis=1)
    monetary_data_noisy[f"dtu_greedy_{noisy}_hb"] = monetary_data_noisy[[f"dtu_greedy_{noisy}_{i}_hb" for i in range(1, noise_samples+1)]].mean(axis=1)
    monetary_data_noisy[f"dtu_greedy_{noisy}_npv"] = monetary_data_noisy[[f"dtu_greedy_{noisy}_{i}_npv" for i in range(1, noise_samples+1)]].mean(axis=1)
    monetary_data_noisy[f"dtu_ta_{noisy}_tb"] = monetary_data_noisy[[f"dtu_ta_{noisy}_{i}_tb" for i in range(1, noise_samples+1)]].mean(axis=1)
    monetary_data_noisy[f"dtu_ta_{noisy}_hb"] = monetary_data_noisy[[f"dtu_ta_{noisy}_{i}_hb" for i in range(1, noise_samples+1)]].mean(axis=1)
    monetary_data_noisy[f"dtu_ta_{noisy}_npv"] = monetary_data_noisy[[f"dtu_ta_{noisy}_{i}_npv" for i in range(1, noise_samples+1)]].mean(axis=1)

In [ ]:
with open(join(paths["output_folder"], tud_save, f"{tud_save}_order_Penalty.json"), "r") as f:
  tud_p_order = json.load(f)
with open(join(paths["output_folder"], tud_save, f"{tud_save}_order_TB.json"), "r") as f:
  tud_tb_order = json.load(f)
with open(join(paths["output_folder"], tud_save, f"{tud_save}_order_TBHB.json"), "r") as f:
  tud_tbhb_order = json.load(f)

with open(join(paths["output_folder"], dtu_save, f"{dtu_save}_order_LPGreedy.json"), "r") as f:
  dtu_greedy_order = json.load(f)
with open(join(paths["output_folder"], dtu_save, f"{dtu_save}_order_TA.json"), "r") as f:
  dtu_ta_order = json.load(f)
  
order_segment = pd.DataFrame({
    "tud_p": tud_p_order,
    "tud_tb": tud_tb_order,
    "tud_tbhb": tud_tbhb_order,
    "dtu_greedy": dtu_greedy_order,
    "dtu_ta": dtu_ta_order.copy().extend((len(dtu_greedy_order)-len(dtu_ta_order))*[np.nan]),
})

In [ ]:
tud_p_order_yearly_raw, tud_tb_order_yearly_raw, tud_tbhb_order_yearly_raw, dtu_greedy_order_raw_yearly, dtu_ta_order_raw_yearly = load_yearly_ordering(tud_save, dtu_save)
order_yearly = pd.DataFrame({
    "year": range(0, 51), 
    "tud_p": dict(sorted(tud_p_order_yearly_raw.items())).values(),
    "tud_tb": dict(sorted(tud_tb_order_yearly_raw.items())).values(),
    "tud_tbhb": dict(sorted(tud_tbhb_order_yearly_raw.items())).values(),
    "dtu_greedy": dict(sorted(dtu_greedy_order_raw_yearly.items())).values(),
    "dtu_ta": dict(sorted(dtu_ta_order_raw_yearly.items())).values(),
})

In [ ]:
tud_p_segments_added = range(0, len(order_yearly["tud_p"].iloc[-1])+1)
tud_tb_segments_added = range(0, len(order_yearly["tud_tb"].iloc[-1])+1)
tud_tbhb_segments_added = range(0, len(order_yearly["tud_tbhb"].iloc[-1])+1)

dtu_ta_segments_added = range(0, len(order_yearly["dtu_ta"].iloc[-1])+1)
dtu_greedy_segments_added = range(0, len(order_yearly["dtu_greedy"].iloc[-1])+1)

In [ ]:
tud_p_yearly_added, tud_p_segment_year = get_yearly_added(tud_p_order_yearly_raw)
tud_tb_yearly_added, tud_tb_segment_year = get_yearly_added(tud_tb_order_yearly_raw)
tud_tbhb_yearly_added, tud_tbhb_segment_year = get_yearly_added(tud_tbhb_order_yearly_raw)

dtu_greedy_yearly_added, dtu_greedy_segment_year = get_yearly_added(dtu_greedy_order_raw_yearly)
dtu_ta_yearly_added, dtu_ta_segment_year = get_yearly_added(dtu_ta_order_raw_yearly)

In [ ]:
dtu_ta_missing_segments = set(dtu_greedy_order) - set(dtu_ta_order)
dtu_ta_missing_segments = {seg: {seg: idx+1 for idx, seg in enumerate(dtu_greedy_order)}[seg] for seg in dtu_ta_missing_segments}
dtu_ta_missing_segments_ordered = list(sorted(dtu_ta_missing_segments, key=dtu_ta_missing_segments.get))

In [ ]:
geom_proposed["Year_tud_p"] = geom_proposed["SegId"].map(tud_p_segment_year)
geom_proposed["Year_tud_tb"] = geom_proposed["SegId"].map(tud_tb_segment_year)
geom_proposed["Year_tud_tbhb"] = geom_proposed["SegId"].map(tud_tbhb_segment_year)

geom_proposed["Year_dtu_greedy"] = geom_proposed["SegId"].map(dtu_greedy_segment_year)
geom_proposed["Year_dtu_ta"] = geom_proposed["SegId"].map(dtu_ta_segment_year)

# Main
## Figure 2 Copenhagen Setting
### Fig. 2a Existing Network

In [ ]:
fig2a, ax2a = plt.subplots(figsize=(5, 7.6), dpi=dpi)
fig2a.tight_layout()

geom_streetnw.plot(ax=ax2a, color=color_street_nw, linewidth=0.1)
geom_existing.plot(ax=ax2a, color=color_exising_sch, linewidth=1)

ax2a.axis("off")

fig2a.savefig(join(figures, "FIG2a.png"))
plt.show()

### Fig. 2b Proposed Expansion

In [ ]:
fig2b, ax2b = plt.subplots(figsize=(5, 7.6), dpi=dpi)
fig2b.tight_layout()

geom_streetnw.plot(ax=ax2b, color=color_street_nw, linewidth=0.1)
geom_existing.plot(ax=ax2b, color=color_exising_sch, linewidth=1)
geom_proposed.plot(ax=ax2b, color="k", linewidth=1)

ax2b.axis("off")

fig2b.savefig(join(figures, "FIG2b.png"))
plt.show()

### Fig. 2c Demand

In [ ]:
fig2c, ax2c = plt.subplots(figsize=(5, 7.6), dpi=dpi)
fig2c.tight_layout()

geom_demand_plot = geom_demand.copy()
conditions = geom_demand_plot["Demand"] < 1000, (geom_demand_plot["Demand"] >= 1000) & (geom_demand_plot["Demand"] < 2500), (geom_demand_plot["Demand"] >= 2500) & (geom_demand_plot["Demand"] < 5000), (geom_demand_plot["Demand"] >= 5000) & (geom_demand_plot["Demand"] < 10000), geom_demand_plot["Demand"] > 10000
choices = "#e6e6e6", "#b3b3b3", "#808080", "#4d4d4d", "#1a1a1a"
geom_demand_plot["plot_color"] = np.select(conditions, choices, default="r")

geom_streetnw.plot(ax=ax2c, color=color_street_nw, linewidth=0.1, zorder=1)
geom_existing.plot(ax=ax2c, color=color_exising_sch, linewidth=1, zorder=2)
geom_proposed.plot(ax=ax2c, color="k", linewidth=1, zorder=3)
xmin, xmax = ax2c.get_xlim()
ymin, ymax = ax2c.get_ylim()
geom_demand_plot.plot(ax=ax2c, color=geom_demand_plot["plot_color"], edgecolor="black", linewidth=0.1, alpha=0.7, zorder=4)
ax2c.set_xlim(xmin, xmax)
ax2c.set_ylim(ymin, ymax)

ax2c.axis("off")

fig2c.savefig(join(figures, "FIG2c.png"))
plt.show()

## Figure 4 Performance Comparison
### Fig. 4a Net Present Value

In [ ]:
fig4a, ax4a = plt.subplots(figsize=figsize, dpi=dpi)

ax4a.plot(monetary_data["t"][:last_year], monetary_data["dtu_greedy_npv"][:last_year], "o", ms=2, c=color_dtu_greedy)
ax4a.plot(monetary_data["t"][:last_year], monetary_data["dtu_ta_npv"][:last_year], "o", ms=2, c=color_dtu_ta)
ax4a.plot(monetary_data["t"][:last_year], monetary_data["tud_p_npv"][:last_year], "o", ms=2, c=color_tud_p)
ax4a.plot(monetary_data["t"][:last_year], monetary_data["tud_tb_npv"][:last_year], "o", ms=2, c=color_tud_tb)
ax4a.plot(monetary_data["t"][:last_year], monetary_data["tud_tbhb_npv"][:last_year], "o", ms=2, c=color_tud_tbhb)

ax4a.set_xlim(0, last_year+1)
#ax4a.set_ylim(-10, 900)

ax4a.tick_params(axis="y", labelsize=tick_fs)
ax4a.tick_params(axis="x", labelsize=tick_fs)

ax4a.set_xlabel("Year", fontsize=axis_fs)
ax4a.set_ylabel("Net present value [b EUR]", fontsize=axis_fs)

ax4a_in = ax4a.inset_axes([0.62, 0.07, 0.35, 0.35])
ax4a_in.axhline(0, color="#808080", linestyle="--", lw=0.5)
ax4a_in.plot(monetary_data["t"][:last_year], 100*(monetary_data["dtu_greedy_npv"][:last_year]-monetary_data["dtu_ta_npv"][:last_year])/monetary_data["dtu_ta_npv"][:last_year], lw=0.5, c=color_dtu_greedy)
ax4a_in.plot(monetary_data["t"][:last_year], 100*(monetary_data["tud_p_npv"][:last_year]-monetary_data["dtu_ta_npv"][:last_year])/monetary_data["dtu_ta_npv"][:last_year], lw=0.5, c=color_tud_p)
ax4a_in.plot(monetary_data["t"][:last_year], 100*(monetary_data["tud_tb_npv"][:last_year]-monetary_data["dtu_ta_npv"][:last_year])/monetary_data["dtu_ta_npv"][:last_year], lw=0.5, c=color_tud_tb)
ax4a_in.plot(monetary_data["t"][:last_year], 100*(monetary_data["tud_tbhb_npv"][:last_year]-monetary_data["dtu_ta_npv"][:last_year])/monetary_data["dtu_ta_npv"][:last_year], lw=0.5, c=color_tud_tbhb)

ax4a_in.set_ylabel(r"$\mathrm{\Delta}$NPV [%]", fontsize=axis_fs-2, labelpad=0)
ax4a_in.set_xlim(0, 50)
ax4a_in.set_ylim(-45, 15)
ax4a_in.xaxis.set_major_locator(ticker.MultipleLocator(10))
#ax4a_in.yaxis.set_major_locator(ticker.MultipleLocator(10))
ax4a_in.tick_params(axis="x", direction="in", labelsize=tick_fs-2, pad=1.5)
ax4a_in.tick_params(axis="y", direction="in", labelsize=tick_fs-2, pad=1.5)

fig4a.savefig(join(figures, f"FIG4a.{file_format}"))

plt.show()

### Fig. 4b Consumer Surplus

In [ ]:
fig4b, ax4b = plt.subplots(figsize=figsize, dpi=dpi)

ax4b.plot(monetary_data["t"][:last_year], monetary_data["dtu_ta_tb"][:last_year], "o", ms=2,  c=color_dtu_ta)
ax4b.plot(monetary_data["t"][:last_year], monetary_data["dtu_greedy_tb"][:last_year], "o", ms=2,  c=color_dtu_greedy)
ax4b.plot(monetary_data["t"][:last_year], monetary_data["tud_p_tb"][:last_year], "o", ms=2,  c=color_tud_p)
ax4b.plot(monetary_data["t"][:last_year], monetary_data["tud_tb_tb"][:last_year], "o", ms=2,  c=color_tud_tb)
ax4b.plot(monetary_data["t"][:last_year], monetary_data["tud_tbhb_tb"][:last_year], "o", ms=2,  c=color_tud_tbhb)

ax4b.set_xlim(0, last_year+1)
#ax4b.set_ylim(-10, 900)

ax4b.tick_params(axis="x", labelsize=tick_fs)
ax4b.tick_params(axis="y", labelsize=tick_fs)

ax4b.set_xlabel("Year", fontsize=axis_fs)
ax4b.set_ylabel("Consumer surplus [b EUR]", fontsize=axis_fs)

ax4b_in = ax4b.inset_axes([0.62, 0.07, 0.35, 0.35])
ax4b_in.axhline(0, color="#808080", linestyle="--", lw=0.5)
ax4b_in.plot(monetary_data["t"][:last_year], 100*(monetary_data["dtu_greedy_tb"][:last_year]-monetary_data["dtu_ta_tb"][:last_year])/monetary_data["dtu_ta_tb"][:last_year], lw=0.5, c=color_dtu_greedy)
ax4b_in.plot(monetary_data["t"][:last_year], 100*(monetary_data["tud_p_tb"][:last_year]-monetary_data["dtu_ta_tb"][:last_year])/monetary_data["dtu_ta_tb"][:last_year], lw=0.5, c=color_tud_p)
ax4b_in.plot(monetary_data["t"][:last_year], 100*(monetary_data["tud_tb_tb"][:last_year]-monetary_data["dtu_ta_tb"][:last_year])/monetary_data["dtu_ta_tb"][:last_year], lw=0.5, c=color_tud_tb)
ax4b_in.plot(monetary_data["t"][:last_year], 100*(monetary_data["tud_tbhb_tb"][:last_year]-monetary_data["dtu_ta_tb"][:last_year])/monetary_data["dtu_ta_tb"][:last_year], lw=0.5, c=color_tud_tbhb)
ax4b_in.set_ylabel(r"$\mathrm{\Delta}$TB [%]", fontsize=axis_fs-2, labelpad=0)
ax4b_in.set_xlim(0, 50)
ax4b_in.set_ylim(-25, 25)
ax4b_in.xaxis.set_major_locator(ticker.MultipleLocator(10))
#ax4b_in.yaxis.set_major_locator(ticker.MultipleLocator(10))
ax4b_in.tick_params(axis="x", direction="in", labelsize=tick_fs-2, pad=1.5)
ax4b_in.tick_params(axis="y", direction="in", labelsize=tick_fs-2, pad=1.5)

fig4b.savefig(join(figures, f"FIG4b.{file_format}"))

plt.show()

### Fig 4c Bikeability

In [ ]:
utility = pd.DataFrame()

utility_raw_tud_p = pd.DataFrame(list(reversed(tud_p_utility_data)))
utility["tud_p"] = utility_raw_tud_p.replace(to_replace=0.0, value=np.nan).dropna()[0].to_list()

utility_raw_tud_tb = pd.DataFrame(list(reversed(tud_tb_utility_data)))
utility["tud_tb"] = utility_raw_tud_tb.replace(to_replace=0.0, value=np.nan).dropna()[0].to_list()

utility_raw_tud_tbhb = pd.DataFrame(list(reversed(tud_tbhb_utility_data)))
utility["tud_tbhb"] = utility_raw_tud_tbhb.replace(to_replace=0.0, value=np.nan).dropna()[0].to_list()

utility_raw_dtu_greedy = pd.DataFrame(list(reversed(dtu_greedy_utility_data)))
utility["dtu_greedy"] = utility_raw_dtu_greedy.replace(to_replace=0.0, value=np.nan).dropna()[0].to_list()

utility_raw_dtu_ta = pd.DataFrame(list(reversed(dtu_ta_utility_data)))
utility_raw_dtu_ta = utility_raw_dtu_ta.replace(to_replace=0.0, value=np.nan).dropna()[0].to_list()
utility_raw_dtu_ta.extend((len(dtu_greedy_order)-len(dtu_ta_order))*[utility_raw_dtu_ta[-1]])
utility["dtu_ta"] = utility_raw_dtu_ta

In [ ]:
max_utility_tud_p = utility["tud_p"].max()
max_utility_tud_tb = utility["tud_tb"].max()
max_utility_tud_tbhb = utility["tud_tbhb"].max()
max_utility_dtu_greedy = utility["dtu_greedy"].max()
max_utility_dtu_ta = utility["dtu_ta"].max()
max_utility = utility[["tud_p", "tud_tb", "tud_tbhb", "dtu_greedy", "dtu_ta"]].max().max()

In [ ]:
min_utility_tud_p = utility["tud_p"].min()
min_utility_tud_tb = utility["tud_tb"].min()
min_utility_tud_tbhb = utility["tud_tbhb"].min()
min_utility_dtu_greedy = utility["dtu_greedy"].min()
min_utility_dtu_ta = utility["dtu_ta"].min()
min_utility = utility[["tud_p", "tud_tb", "tud_tbhb", "dtu_greedy", "dtu_ta"]].min().min()

In [ ]:
ba_segment = pd.DataFrame()
ba_segment["tud_p"] = [(min_utility - i) / (min_utility - max_utility) for i in utility["tud_p"]]
ba_segment["tud_tb"] = [(min_utility - i) / (min_utility - max_utility) for i in utility["tud_tb"]]
ba_segment["tud_tbhb"] = [(min_utility - i) / (min_utility - max_utility) for i in utility["tud_tbhb"]]
ba_segment["dtu_greedy"] = [(min_utility - i) / (min_utility - max_utility) for i in utility["dtu_greedy"]]
ba_segment["dtu_ta"] = [(min_utility - i) / (min_utility - max_utility) for i in utility["dtu_ta"]]

ba_year = pd.DataFrame({"t": years_to_plot})
ba_year["tud_p"] = [ba_segment["tud_p"].to_list()[len(order_yearly.iloc[year]["tud_p"])] for year in ba_year["t"]]
ba_year["tud_tb"] = [ba_segment["tud_tb"].to_list()[len(order_yearly.iloc[year]["tud_tb"])] for year in ba_year["t"]]
ba_year["tud_tbhb"] = [ba_segment["tud_tbhb"].to_list()[len(order_yearly.iloc[year]["tud_tbhb"])] for year in ba_year["t"]]
ba_year["dtu_greedy"] = [ba_segment["dtu_greedy"].to_list()[len(order_yearly.iloc[year]["dtu_greedy"])] for year in ba_year["t"]]
ba_year["dtu_ta"] = [ba_segment["dtu_ta"].to_list()[len(order_yearly.iloc[year]["dtu_ta"])] for year in ba_year["t"]]

In [ ]:
fig4c, ax4c = plt.subplots(figsize=figsize, dpi=dpi)

ax4c.plot(ba_year["t"][:last_year+1], ba_year["dtu_ta"][:last_year+1], lw=1, c=color_dtu_ta)
ax4c.plot(ba_year["t"][:last_year+1], ba_year["dtu_greedy"][:last_year+1], lw=1, c=color_dtu_greedy)
ax4c.plot(ba_year["t"][:last_year+1], ba_year["tud_p"][:last_year+1], lw=1, c=color_tud_p)
ax4c.plot(ba_year["t"][:last_year+1], ba_year["tud_tb"][:last_year+1], lw=1, c=color_tud_tb)
ax4c.plot(ba_year["t"][:last_year+1], ba_year["tud_tbhb"][:last_year+1], lw=1, c=color_tud_tbhb)

ax4c.xaxis.set_major_locator(ticker.MultipleLocator(10))
ax4c.set_xlim(0, last_year)
ax4c.set_ylim(0, 1)

ax4c.tick_params(axis="y", labelsize=tick_fs)
ax4c.tick_params(axis="x", labelsize=tick_fs)

ax4c.set_xlabel("Year", fontsize=axis_fs)
ax4c.set_ylabel("Bikeability", fontsize=axis_fs)

ax4c_in = ax4c.inset_axes([0.35, 0.13, 0.6, 0.6])
ax4c_in.plot(dtu_ta_segments_added, ba_segment["dtu_ta"][:len(dtu_ta_segments_added)], lw=1, c=color_dtu_ta)
ax4c_in.plot(dtu_greedy_segments_added, ba_segment["dtu_greedy"], lw=1, c=color_dtu_greedy)
ax4c_in.plot(tud_p_segments_added, ba_segment["tud_p"], lw=1, c=color_tud_p)
ax4c_in.plot(tud_tb_segments_added, ba_segment["tud_tb"], lw=1, c=color_tud_tb)
ax4c_in.plot(tud_tbhb_segments_added, ba_segment["tud_tbhb"], lw=1, c=color_tud_tbhb)

ax4c_in.set_xlim(0, 202)
ax4c_in.set_ylim(0, 1)

ax4c_in.xaxis.set_major_locator(ticker.MultipleLocator(50))
ax4c_in.yaxis.set_major_locator(ticker.MultipleLocator(0.2))
ax4c_in.tick_params(axis="y", direction="in", labelsize=tick_fs-1, pad=1.5)
ax4c_in.tick_params(axis="x", direction="in", labelsize=tick_fs-1, pad=1.5)

ax4c_in.set_xlabel("Number of segments added", fontsize=axis_fs-2, labelpad=0)
#ax4c_in.set_ylabel("Bikeability", fontsize=axis_fs-2, labelpad=0)

fig4c.savefig(join(figures, f"FIG4c.{file_format}"))

plt.show()

## Figure 5 Planned Networks
### Fig. 5a 3 Years

In [ ]:
fig5a, ax5a = plt.subplots(figsize=(5, 7.6), dpi=dpi)
fig5a.tight_layout()

plot_year = 3

geom_proposed_plot = geom_proposed.copy()
geom_proposed_plot.fillna({f"Year_{dtu_comp}": 100}, inplace=True)
conditions = (geom_proposed_plot[f"Year_{tud_comp}"] <= plot_year) & (geom_proposed_plot[f"Year_{dtu_comp}"] <= plot_year), (geom_proposed_plot[f"Year_{tud_comp}"] <= plot_year) & (geom_proposed_plot[f"Year_{dtu_comp}"] > plot_year), (geom_proposed_plot[f"Year_{tud_comp}"] > plot_year) & (geom_proposed_plot[f"Year_{dtu_comp}"] <= plot_year)
choices = color_combined, color[f"color_{tud_comp}"], color[f"color_{dtu_comp}"]
geom_proposed_plot["plot_color"] = np.select(conditions, choices, default=color_upgradable_sch)
choices = 3, 1, 1
geom_proposed_plot["plot_zorder"] = np.select(conditions, choices, default=0)

geom_streetnw.plot(ax=ax5a, color=color_street_nw, linewidth=0.1)
geom_existing.plot(ax=ax5a, color=color_exising_sch, linewidth=1)
geom_proposed_plot.sort_values(by=["plot_zorder"]).plot(ax=ax5a, color=geom_proposed_plot["plot_color"], linewidth=1)

ax5a.axis("off")

fig5a.savefig(join(figures, f"FIG5a.png"))
plt.show()

In [ ]:
fig5b, ax5b = plt.subplots(figsize=(5, 7.6), dpi=dpi)
fig5b.tight_layout()

plot_year = 10

geom_proposed_plot = geom_proposed.copy()
geom_proposed_plot.fillna({f"Year_{dtu_comp}": 100}, inplace=True)
conditions = (geom_proposed_plot[f"Year_{tud_comp}"] <= plot_year) & (geom_proposed_plot[f"Year_{dtu_comp}"] <= plot_year), (geom_proposed_plot[f"Year_{tud_comp}"] <= plot_year) & (geom_proposed_plot[f"Year_{dtu_comp}"] > plot_year), (geom_proposed_plot[f"Year_{tud_comp}"] > plot_year) & (geom_proposed_plot[f"Year_{dtu_comp}"] <= plot_year)
choices = color_combined, color[f"color_{tud_comp}"], color[f"color_{dtu_comp}"]
geom_proposed_plot["plot_color"] = np.select(conditions, choices, default="#ebe1d7")
choices = 3, 1, 1
geom_proposed_plot["plot_zorder"] = np.select(conditions, choices, default=0)

geom_streetnw.plot(ax=ax5b, color=color_street_nw, linewidth=0.1)
geom_existing.plot(ax=ax5b, color=color_exising_sch, linewidth=1)
geom_proposed_plot.sort_values(by=["plot_zorder"]).plot(ax=ax5b, color=geom_proposed_plot["plot_color"], linewidth=1)

ax5b.axis("off")

fig5b.savefig(join(figures, f"FIG5b.png"))
plt.show()

In [ ]:
fig5c, ax5c = plt.subplots(figsize=(5, 7.6), dpi=dpi)
fig5c.tight_layout()

plot_year = 30

geom_proposed_plot = geom_proposed.copy()
geom_proposed_plot.fillna({f"Year_{dtu_comp}": 100}, inplace=True)
conditions = (geom_proposed_plot[f"Year_{tud_comp}"] <= plot_year) & (geom_proposed_plot[f"Year_{dtu_comp}"] <= plot_year), (geom_proposed_plot[f"Year_{tud_comp}"] <= plot_year) & (geom_proposed_plot[f"Year_{dtu_comp}"] > plot_year), (geom_proposed_plot[f"Year_{tud_comp}"] > plot_year) & (geom_proposed_plot[f"Year_{dtu_comp}"] <= plot_year)
choices = color_combined, color[f"color_{tud_comp}"], color[f"color_{dtu_comp}"]
geom_proposed_plot["plot_color"] = np.select(conditions, choices, default="#ebe1d7")
choices = 3, 4, 1
geom_proposed_plot["plot_zorder"] = np.select(conditions, choices, default=0)

geom_streetnw.plot(ax=ax5c, color=color_street_nw, linewidth=0.1)
geom_existing.plot(ax=ax5c, color=color_exising_sch, linewidth=1)
geom_proposed_plot.sort_values(by=["plot_zorder"]).plot(ax=ax5c, color=geom_proposed_plot["plot_color"], linewidth=1)

ax5c.axis("off")

fig5c.savefig(join(figures, f"FIG5c.png"))
plt.show()

### Observables at Comparison Points

In [ ]:
dtu_comp_points = pd.DataFrame({
    "t": monetary_data["t"].iloc[[2, 9, 29, 49]],
    "length" : [sum([segment_cost.loc[segment_cost["SegmentId"] == seg_id, "LengthGraph"].iloc[0] for seg_id in order_yearly[dtu_comp][year]]) for year in [3, 10, 30, 50]],
    "length_rel" : [sum([segment_cost.loc[segment_cost["SegmentId"] == seg_id, "LengthGraph"].iloc[0] / total_segment_length for seg_id in order_yearly[dtu_comp][year]]) for year in [3, 10, 30, 50]],
    "CC_rel": [sum([segment_cost.loc[segment_cost["SegmentId"] == seg_id, "ConstructionCosts"].iloc[0] / total_segment_cost for seg_id in order_yearly[dtu_comp][year]]) for year in [3, 10, 30, 50]],
    "NPV": 1000*monetary_data[f"{dtu_comp}_npv"].iloc[[2, 9, 29, 49]],
    "TB": 1000*monetary_data[f"{dtu_comp}_tb"].iloc[[2, 9, 29, 49]],
    "HB": 1000*monetary_data[f"{dtu_comp}_hb"].iloc[[2, 9, 29, 49]],
    "CC": [sum([segment_cost.loc[segment_cost["SegmentId"] == seg_id, "ConstructionCosts"].iloc[0] for seg_id in order_yearly[dtu_comp][year]]) for year in [3, 10, 30, 50]],
    "MC": [sum([segment_cost.loc[segment_cost["SegmentId"] == seg_id, "MaintenanceCosts"].iloc[0] for seg_id in order_yearly[dtu_comp][year-1]]) for year in [3, 10, 30, 50]],
})

In [ ]:
tud_comp_points = pd.DataFrame({
    "t": monetary_data["t"].iloc[[2, 9, 29, 49]],
    "length" : [sum([segment_cost.loc[segment_cost["SegmentId"] == seg_id, "LengthGraph"].iloc[0] for seg_id in order_yearly[tud_comp][year]]) for year in [3, 10, 30, 50]],
    "length_rel" : [sum([segment_cost.loc[segment_cost["SegmentId"] == seg_id, "LengthGraph"].iloc[0] / total_segment_length for seg_id in order_yearly[tud_comp][year]]) for year in [3, 10, 30, 50]],
    "CC_rel": [sum([segment_cost.loc[segment_cost["SegmentId"] == seg_id, "ConstructionCosts"].iloc[0] / total_segment_cost for seg_id in order_yearly[tud_comp][year]]) for year in [3, 10, 30, 50]],
    "NPV": 1000*monetary_data[f"{tud_comp}_npv"].iloc[[2, 9, 29, 49]],
    "TB": 1000*monetary_data[f"{tud_comp}_tb"].iloc[[2, 9, 29, 49]],
    "HB": 1000*monetary_data[f"{tud_comp}_hb"].iloc[[2, 9, 29, 49]],
    "CC": [sum([segment_cost.loc[segment_cost["SegmentId"] == seg_id, "ConstructionCosts"].iloc[0] for seg_id in order_yearly[tud_comp][year]]) for year in [3, 10, 30, 50]],
    "MC": [sum([segment_cost.loc[segment_cost["SegmentId"] == seg_id, "MaintenanceCosts"].iloc[0] for seg_id in order_yearly[tud_comp][year-1]]) for year in [3, 10, 30, 50]],
})

In [ ]:
comp_points = dtu_comp_points.transpose().rename(columns={2: "DO Year 3", 9: "DO Year 10", 29: "DO Year 30", 49: "DO Year 50"})
comp_points.insert(1, "BP Year 3", tud_comp_points.transpose()[2])
comp_points.insert(3, "BP Year 10", tud_comp_points.transpose()[9])
comp_points.insert(5, "BP Year 30", tud_comp_points.transpose()[29])
comp_points.insert(7, "BP Year 50", tud_comp_points.transpose()[49])
comp_points.rename(columns={2: "DO Year 3", 9: "DO Year 10", 29: "DO Year 30", 49: "DO Year 50"})
comp_points.drop(index="t", inplace=True)
comp_points

In [ ]:
comp_rel = pd.DataFrame({
    "length" : (tud_comp_points["length"] - dtu_comp_points["length"]) / dtu_comp_points["length"],
    "length_rel" : (tud_comp_points["length_rel"] - dtu_comp_points["length_rel"]) / dtu_comp_points["length_rel"],
    "CC_rel": (tud_comp_points["CC_rel"] - dtu_comp_points["CC_rel"]) / dtu_comp_points["CC_rel"],
    "NPV": (tud_comp_points["NPV"] - dtu_comp_points["NPV"]) / dtu_comp_points["NPV"],
    "TB": (tud_comp_points["TB"] - dtu_comp_points["TB"]) / dtu_comp_points["TB"],
    "HB": (tud_comp_points["HB"] - dtu_comp_points["HB"]) / dtu_comp_points["HB"],
    "CC": (tud_comp_points["CC"] - dtu_comp_points["CC"]) / dtu_comp_points["CC"],
    "MC": (tud_comp_points["MC"] - dtu_comp_points["MC"]) / dtu_comp_points["MC"],
})
comp_rel = comp_rel.transpose().rename(columns={2: "Year 3", 9: "Year 10", 29: "Year 30", 49: "Year 50"})
comp_rel

## Figure 6 Build Order Comparison
### Fig. 6a Rank Priority

In [ ]:
ordering = pd.DataFrame({"segment": sorted(dtu_greedy_order)})
ordering_missing = pd.DataFrame({"segment": sorted(dtu_ta_missing_segments_ordered)})

ordering["segment_pos_dtu_greedy"] = ordering["segment"].map({seg: idx+1 for idx, seg in enumerate(dtu_greedy_order)})
ordering_missing["segment_pos_dtu_greedy"] = ordering_missing["segment"].map({seg: idx+1 for idx, seg in enumerate(dtu_greedy_order)})
ordering["segment_pos_dtu_ta"] = ordering["segment"].map({seg: idx+1 for idx, seg in enumerate(dtu_ta_order)})
ordering_missing["segment_pos_dtu_ta"] = ordering_missing["segment"].map({seg: idx+len(dtu_ta_order)+1 for idx, seg in enumerate(dtu_ta_missing_segments_ordered)})

ordering["segment_pos_tud_p"] = ordering["segment"].map({seg: idx+1 for idx, seg in enumerate(tud_p_order)})
ordering_missing["segment_pos_tud_p"] = ordering_missing["segment"].map({seg: idx+1 for idx, seg in enumerate(tud_p_order)})
ordering["segment_pos_tud_tb"] = ordering["segment"].map({seg: idx+1 for idx, seg in enumerate(tud_tb_order)})
ordering_missing["segment_pos_tud_tb"] = ordering_missing["segment"].map({seg: idx+1 for idx, seg in enumerate(tud_tb_order)})
ordering["segment_pos_tud_tbhb"] = ordering["segment"].map({seg: idx+1 for idx, seg in enumerate(tud_tbhb_order)})
ordering_missing["segment_pos_tud_tbhb"] = ordering_missing["segment"].map({seg: idx+1 for idx, seg in enumerate(tud_tbhb_order)})

ordering["segment_year_dtu_greedy"] = ordering["segment"].map(dtu_greedy_segment_year)
ordering_missing["segment_year_dtu_greedy"] = ordering_missing["segment"].map(dtu_greedy_segment_year)
ordering["segment_year_dtu_ta"] = ordering["segment"].map(dtu_ta_segment_year)
ordering_missing["segment_year_dtu_ta"] = ordering_missing["segment"].map(dtu_ta_segment_year)

ordering["segment_year_tud_p"] = ordering["segment"].map(tud_p_segment_year)
ordering_missing["segment_year_tud_p"] = ordering_missing["segment"].map(tud_p_segment_year)
ordering["segment_year_tud_tb"] = ordering["segment"].map(tud_tb_segment_year)
ordering_missing["segment_year_tud_tb"] = ordering_missing["segment"].map(tud_tb_segment_year)
ordering["segment_year_tud_tbhb"] = ordering["segment"].map(tud_tbhb_segment_year)
ordering_missing["segment_year_tud_tbhb"] = ordering_missing["segment"].map(tud_tbhb_segment_year)

ordering["diff_dtu_greedy"] = ordering["segment_pos_dtu_ta"] - ordering["segment_pos_dtu_greedy"]
ordering_missing["diff_dtu_greedy"] = ordering_missing["segment_pos_dtu_ta"] - ordering_missing["segment_pos_dtu_greedy"]

ordering["diff_tud_p"] = ordering["segment_pos_dtu_ta"] - ordering["segment_pos_tud_p"]
ordering_missing["diff_tud_p"] = ordering_missing["segment_pos_dtu_ta"] - ordering_missing["segment_pos_tud_p"]
ordering["diff_tud_tb"] = ordering["segment_pos_dtu_ta"] - ordering["segment_pos_tud_tb"]
ordering_missing["diff_tud_tb"] = ordering_missing["segment_pos_dtu_ta"] - ordering_missing["segment_pos_tud_tb"]
ordering["diff_tud_tbhb"] = ordering["segment_pos_dtu_ta"] - ordering["segment_pos_tud_tbhb"]
ordering_missing["diff_tud_tbhb"] = ordering_missing["segment_pos_dtu_ta"] - ordering_missing["segment_pos_tud_tbhb"]

ordering["diff_year_dtu_greedy"] = ordering["segment_year_dtu_ta"] - ordering["segment_year_dtu_greedy"]
ordering_missing["diff_year_dtu_greedy"] = ordering_missing["segment_year_dtu_ta"] - ordering_missing["segment_year_dtu_greedy"]

ordering["diff_year_tud_p"] = ordering["segment_year_dtu_ta"] - ordering["segment_year_tud_p"]
ordering_missing["diff_year_tud_p"] = ordering_missing["segment_year_dtu_ta"] - ordering_missing["segment_year_tud_p"]
ordering["diff_year_tud_tb"] = ordering["segment_year_dtu_ta"] - ordering["segment_year_tud_tb"]
ordering_missing["diff_year_tud_tb"] = ordering_missing["segment_year_dtu_ta"] - ordering_missing["segment_year_tud_tb"]
ordering["diff_year_tud_tbhb"] = ordering["segment_year_dtu_ta"] - ordering["segment_year_tud_tbhb"]
ordering_missing["diff_year_tud_tbhb"] = ordering_missing["segment_year_dtu_ta"] - ordering_missing["segment_year_tud_tbhb"]

In [ ]:
fig6a, ax6a = plt.subplots(figsize=(4, 4), dpi=dpi)

ordering_plot = ordering[~ordering.isnull().any(axis=1)].copy()
ordering_missing_plot = ordering_missing.copy()

divnorm = colors.Normalize(vmin=-outlier_threshold, vmax=outlier_threshold, clip=True)
#cmap = colors.LinearSegmentedColormap.from_list("RedDarkBlue", ((0.000, "#d1483a"), (0.500, "#282322"), (1.000, "#4479bb")))
cmap = colors.LinearSegmentedColormap.from_list("DTUDarkTUD", ((0.000, color[f"color_{dtu_comp}"]), (0.500, color_combined), (1.000, color[f"color_{tud_comp}"])))

segment_ordering_outliers = {int(row["segment"]): int(row[f"diff_{tud_comp}"]) for index, row in ordering_plot.iterrows() if abs(row[f"diff_{tud_comp}"]) >= outlier_threshold}
print(segment_ordering_outliers)
segment_ordering_outliers_color = dict()
for seg in ordering["segment"]:
    if seg in segment_ordering_outliers.keys():
        segment_ordering_outliers_color[seg] = cmap(divnorm(ordering_plot[ordering_plot["segment"] == seg][f"diff_{tud_comp}"].iloc[0]))
    else:
        segment_ordering_outliers_color[seg] = "#808080"
segment_year_outliers = {int(row["segment"]): int(row[f"diff_year_{tud_comp}"]) for index, row in ordering_plot.iterrows() if int(row["segment"]) in segment_ordering_outliers.keys()}
print({k: segment_year_outliers[k] for k, v in segment_ordering_outliers.items()})
segment_ordering_color = dict()
segment_ordering_color_only = dict()
for seg in ordering["segment"]:
    if seg in ordering_plot["segment"].to_list():
        segment_ordering_color[seg] = cmap(divnorm(ordering_plot[ordering_plot["segment"] == seg][f"diff_{tud_comp}"].iloc[0]))
        segment_ordering_color_only[seg] = "#808080"
    else:
        segment_ordering_color[seg] = color[f"color_{tud_comp}"]
        segment_ordering_color_only[seg] = color[f"color_{tud_comp}"]

ax6a.plot(np.arange(0, 204), np.arange(0, 204), "--", c="#808080", alpha=0.75, zorder=2)
ax6a.plot(np.arange(0, 204), np.arange(0, 204)+outlier_threshold, "--", c="#808080", alpha=0.5, lw=0.5, zorder=2)
ax6a.plot(np.arange(0, 204), np.arange(0, 204)-outlier_threshold, "--", c="#808080", alpha=0.5, lw=0.5, zorder=2)
ax6a.scatter(ordering_missing_plot[f"segment_pos_{dtu_comp}"], ordering_missing_plot[f"segment_pos_{tud_comp}"], s=15, c="#808080", alpha=0.4, linewidths=0, zorder=3)
scatter = ax6a.scatter(ordering_plot[f"segment_pos_{dtu_comp}"], ordering_plot[f"segment_pos_{tud_comp}"], s=15, c=ordering_plot[f"diff_{tud_comp}"], norm=divnorm, cmap=cmap, linewidths=0, zorder=3)

ax6a.set_xlim(0, 203)
ax6a.set_ylim(0, 203)
ax6a.set_aspect("equal")

ax6a.tick_params(axis="y", labelsize=tick_fs)
ax6a.tick_params(axis="x", labelsize=tick_fs)

ax6a.set_xlabel("Rank in direct optimization", fontsize=axis_fs)
ax6a.set_ylabel("Rank in backwards percolation", fontsize=axis_fs)

fig6a.savefig(join(figures, f"FIG6a.{file_format}"), bbox_inches="tight")

plt.show()

cbar = fig6a.colorbar(scatter, ax=ax6a, orientation="vertical", ticks=[-outlier_threshold, -int(outlier_threshold/2), 0, int(outlier_threshold/2), outlier_threshold])
cbar.ax.set_ylabel(r"Ranking difference $\Delta \mathrm{RANK}$", fontsize=8)
cbar.ax.set_yticklabels([f"-{outlier_threshold}", f"-{int(outlier_threshold/2)}", "0", f"{int(outlier_threshold/2)}", f"{outlier_threshold}"])
cbar.ax.tick_params(labelsize=tick_fs)
ax6a.remove()
fig6a.savefig(join(figures, f"FIG6a_cbar.{file_format}"), bbox_inches="tight")

### Fig. 6b Rank Priority Map

In [ ]:
fig6b, ax6b = plt.subplots(figsize=(5, 7.6), dpi=dpi)
fig6b.tight_layout()

geom_proposed_plot = geom_proposed.copy()
geom_proposed_plot["color"] = geom_proposed_plot["SegId"].map(segment_ordering_color)

geom_streetnw.plot(ax=ax6b, color=color_street_nw, linewidth=0.1)
geom_existing.plot(ax=ax6b, color=color_exising_sch, linewidth=1)
geom_proposed_plot.plot(ax=ax6b, color=geom_proposed_plot["color"], linewidth=1)

ax6b.axis("off")

fig6b.savefig(join(figures, f"FIG6b.png"))
plt.show()

In [ ]:
fig6b_helper, ax6b_helper = plt.subplots(1, 2, figsize=(10, 7.6), dpi=dpi)
fig6b_helper.tight_layout()

geom_proposed_plot = geom_proposed.copy()
geom_proposed_plot["color"] = geom_proposed_plot["SegId"].map(segment_ordering_outliers_color)
geom_streetnw.plot(ax=ax6b_helper[0], color=color_street_nw, linewidth=0.1)
geom_existing.plot(ax=ax6b_helper[0], color=color_exising_sch, linewidth=1)
geom_proposed_plot.loc[geom_proposed_plot["color"] != "#808080", "plot_zorder"] = 1
geom_proposed_plot.fillna({"plot_zorder": 0}, inplace=True)
geom_proposed_plot.sort_values(by=["plot_zorder"]).plot(ax=ax6b_helper[0], color=geom_proposed_plot["color"], linewidth=1)
ax6b_helper[0].set_title("Outliers", fontsize=axis_fs)

geom_proposed_plot = geom_proposed.copy()
geom_proposed_plot["color"] = geom_proposed_plot["SegId"].map(segment_ordering_color_only)
geom_streetnw.plot(ax=ax6b_helper[1], color=color_street_nw, linewidth=0.1)
geom_existing.plot(ax=ax6b_helper[1], color=color_exising_sch, linewidth=1)
geom_proposed_plot.loc[geom_proposed_plot["color"] != "#808080", "plot_zorder"] = 1
geom_proposed_plot.fillna({"plot_zorder": 0}, inplace=True)
geom_proposed_plot.sort_values(by=["plot_zorder"]).plot(ax=ax6b_helper[1], color=geom_proposed_plot["color"], linewidth=1)
ax6b_helper[1].set_title("Only build in BP", fontsize=axis_fs)

ax6b_helper[0].axis("off")
ax6b_helper[1].axis("off")

fig6b_helper.savefig(join(figures, f"FIG6b_helper.png"))
plt.show()

## Figure 7 Robustness
### Fig. 7a Change in NPV for noisy demand

In [ ]:
fig7a, ax7a = plt.subplots(figsize=figsize, dpi=dpi)

ax7a.axhline(0, color="#808080", linestyle="--", lw=1)
ax7a.plot(monetary_data_noisy["t"][:last_year], 1000*(monetary_data_noisy["dtu_ta_noisy_demand_npv"][:last_year]-monetary_data["dtu_ta_npv"][:last_year]), lw=1.0, c=color_dtu_ta)
ax7a.plot(monetary_data_noisy["t"][:last_year], 1000*(monetary_data_noisy["dtu_greedy_noisy_demand_npv"][:last_year]-monetary_data["dtu_greedy_npv"][:last_year]), lw=1.0, c=color_dtu_greedy)
ax7a.plot(monetary_data_noisy["t"][:last_year], 1000*(monetary_data_noisy["tud_p_noisy_demand_npv"][:last_year]-monetary_data["tud_p_npv"][:last_year]), lw=1.0, c=color_tud_p)
ax7a.plot(monetary_data_noisy["t"][:last_year], 1000*(monetary_data_noisy["tud_tb_noisy_demand_npv"][:last_year]-monetary_data["tud_tb_npv"][:last_year]), lw=1.0, c=color_tud_tb)
ax7a.plot(monetary_data_noisy["t"][:last_year], 1000*(monetary_data_noisy["tud_tbhb_noisy_demand_npv"][:last_year]-monetary_data["tud_tbhb_npv"][:last_year]), lw=1.0, c=color_tud_tbhb)

ax7a.set_xlim(0, last_year)
ax7a.set_ylim(-40, 40)

ax7a.tick_params(axis="y", labelsize=tick_fs)
ax7a.tick_params(axis="x", labelsize=tick_fs)

ax7a.set_xlabel("Year", fontsize=axis_fs)
ax7a.set_ylabel("Change in NPV [m EUR]", fontsize=axis_fs)

ax7a_in = ax7a.inset_axes([0.62, 0.08, 0.33, 0.33])
for i in range(1, noise_samples+1):
    ax7a_in.plot(monetary_data_noisy["t"][:last_year], 1000*(monetary_data_noisy[f"dtu_ta_noisy_demand_{i}_npv"][:last_year]-monetary_data["dtu_ta_npv"][:last_year]), lw=0.5, c=color_dtu_ta, alpha=0.2)
ax7a_in.plot(monetary_data_noisy["t"][:last_year], 1000*(monetary_data_noisy["dtu_ta_noisy_demand_npv"][:last_year]-monetary_data["dtu_ta_npv"][:last_year]), lw=0.5, c=color_dtu_ta)

"""print(f"TA noisy demand: {total_deviation(1000*monetary_data_noisy["dtu_ta_noisy_demand_npv"][:last_year], 1000*monetary_data["dtu_ta_npv"][:last_year], monetary_data_noisy["t"][:last_year])}")
print(f"Greedy noisy demand: {total_deviation(1000*monetary_data_noisy["dtu_greedy_noisy_demand_npv"][:last_year], 1000*monetary_data["dtu_greedy_npv"][:last_year], monetary_data_noisy["t"][:last_year])}")
print(f"Penalty noisy demand: {total_deviation(1000*monetary_data_noisy["p_noisy_demand_npv"][:last_year], 1000*monetary_data["tud_p_npv"][:last_year], monetary_data_noisy["t"][:last_year])}")
print(f"TB noisy demand: {total_deviation(1000*monetary_data_noisy["cs_noisy_demand_npv"][:last_year], 1000*monetary_data["tud_tb_npv"][:last_year], monetary_data_noisy["t"][:last_year])}")
print(f"TBHB noisy demand: {total_deviation(1000*monetary_data_noisy["cshb_noisy_demand_npv"][:last_year], 1000*monetary_data["tud_tbhb_npv"][:last_year], monetary_data_noisy["t"][:last_year])}")"""

#ax7a_in.set_ylabel("NPV [b EUR]", fontsize=axis_fs-2, labelpad=0)
ax7a_in.set_xlim(0, last_year)
ax7a_in.set_ylim(-10, 10)
ax7a_in.xaxis.set_major_locator(ticker.MultipleLocator(10))
ax7a_in.yaxis.set_major_locator(ticker.MultipleLocator(5))
ax7a_in.tick_params(axis="x", direction="in", labelsize=tick_fs-1, pad=1.5)
ax7a_in.tick_params(axis="y", direction="in", labelsize=tick_fs-1, pad=1.5)

fig7a.savefig(join(figures, f"FIG7a.{file_format}"))

plt.show()

## Fig. 7b Change in NPV for noisy costs

In [ ]:
fig7b, ax7b = plt.subplots(figsize=figsize, dpi=dpi)

ax7b.axhline(0, color="#808080", linestyle="--", lw=1)
ax7b.plot(monetary_data_noisy["t"][:last_year], 1000*(monetary_data_noisy["dtu_ta_noisy_costs_npv"][:last_year]-monetary_data["dtu_ta_npv"][:last_year]), lw=1.0, c=color_dtu_ta)
ax7b.plot(monetary_data_noisy["t"][:last_year], 1000*(monetary_data_noisy["dtu_greedy_noisy_costs_npv"][:last_year]-monetary_data["dtu_greedy_npv"][:last_year]), lw=1.0, c=color_dtu_greedy)
ax7b.plot(monetary_data_noisy["t"][:last_year], 1000*(monetary_data_noisy["tud_p_noisy_costs_npv"][:last_year]-monetary_data["tud_p_npv"][:last_year]), lw=1.0, c=color_tud_p)
ax7b.plot(monetary_data_noisy["t"][:last_year], 1000*(monetary_data_noisy["tud_tb_noisy_costs_npv"][:last_year]-monetary_data["tud_tb_npv"][:last_year]), lw=1.0, c=color_tud_tb)
ax7b.plot(monetary_data_noisy["t"][:last_year], 1000*(monetary_data_noisy["tud_tbhb_noisy_costs_npv"][:last_year]-monetary_data["tud_tbhb_npv"][:last_year]), lw=1.0, c=color_tud_tbhb)


ax7b.set_xlim(0, last_year)
ax7b.set_ylim(-120, 20)

ax7b.tick_params(axis="y", labelsize=tick_fs)
ax7b.tick_params(axis="x", labelsize=tick_fs)

ax7b.set_xlabel("Year", fontsize=axis_fs)
ax7b.set_ylabel("Change in NPV [m EUR]", fontsize=axis_fs)

ax7b_in = ax7b.inset_axes([0.62, 0.08, 0.33, 0.33])
for i in range(1, noise_samples+1):
    ax7b_in.plot(monetary_data_noisy["t"][:last_year], 1000*(monetary_data_noisy[f"dtu_ta_noisy_costs_{i}_npv"][:last_year]-monetary_data["dtu_ta_npv"][:last_year]), lw=0.5, c=color_dtu_ta, alpha=0.2)
ax7b_in.plot(monetary_data_noisy["t"][:last_year], 1000*(monetary_data_noisy["dtu_ta_noisy_costs_npv"][:last_year]-monetary_data["dtu_ta_npv"][:last_year]), lw=0.5, c=color_dtu_ta)

"""print(f"TA noisy costs: {total_deviation(1000*monetary_data_noisy["dtu_ta_noisy_costs_npv"][:last_year], 1000*monetary_data["dtu_ta_npv"][:last_year], monetary_data_noisy["t"][:last_year])}")
print(f"Greedy noisy costs: {total_deviation(1000*monetary_data_noisy["dtu_greedy_noisy_costs_npv"][:last_year], 1000*monetary_data["dtu_greedy_npv"][:last_year], monetary_data_noisy["t"][:last_year])}")
print(f"Penalty noisy costs: {total_deviation(1000*monetary_data_noisy["p_noisy_costs_npv"][:last_year], 1000*monetary_data["tud_p_npv"][:last_year], monetary_data_noisy["t"][:last_year])}")
print(f"TB noisy costs: {total_deviation(1000*monetary_data_noisy["cs_noisy_costs_npv"][:last_year], 1000*monetary_data["tud_tb_npv"][:last_year], monetary_data_noisy["t"][:last_year])}")
print(f"TBHB noisy costs: {total_deviation(1000*monetary_data_noisy["cshb_noisy_costs_npv"][:last_year], 1000*monetary_data["tud_tbhb_npv"][:last_year], monetary_data_noisy["t"][:last_year])}")"""

#ax7b_in.set_ylabel("NPV [b EUR]", fontsize=axis_fs-2, labelpad=0)
ax7b_in.set_xlim(0, last_year)
ax7b_in.set_ylim(-260, 25)
ax7b_in.xaxis.set_major_locator(ticker.MultipleLocator(10))
#ax7b_in.yaxis.set_major_locator(ticker.MultipleLocator(1))
ax7b_in.tick_params(axis="x", direction="in", labelsize=tick_fs-1, pad=1.5)
ax7b_in.tick_params(axis="y", direction="in", labelsize=tick_fs-1, pad=1.5)

fig7b.savefig(join(figures, f"FIG7b.{file_format}"))

plt.show()

In [ ]:
print(1000*max([max(abs(monetary_data_noisy[f'dtu_ta_noisy_costs_{i}_npv'][:last_year]-monetary_data["dtu_ta_npv"][:last_year])) for i in range(1, noise_samples+1)]))

### Fig. 7c Change in NPV for noisy velocities

In [ ]:
fig7c, ax7c = plt.subplots(figsize=figsize, dpi=dpi)

ax7c.axhline(0, color="#808080", linestyle="--", lw=1)
ax7c.plot(monetary_data_noisy["t"][:last_year], 1000*(monetary_data_noisy["dtu_ta_noisy_speeds_npv"][:last_year]-monetary_data["dtu_ta_npv"][:last_year]), lw=1.0, c=color_dtu_ta)
ax7c.plot(monetary_data_noisy["t"][:last_year], 1000*(monetary_data_noisy["dtu_greedy_noisy_speeds_npv"][:last_year]-monetary_data["dtu_greedy_npv"][:last_year]), lw=1.0, c=color_dtu_greedy)
ax7c.plot(monetary_data_noisy["t"][:last_year], 1000*(monetary_data_noisy["tud_p_noisy_speeds_npv"][:last_year]-monetary_data["tud_p_npv"][:last_year]), lw=1.0, c=color_tud_p)
ax7c.plot(monetary_data_noisy["t"][:last_year], 1000*(monetary_data_noisy["tud_tb_noisy_speeds_npv"][:last_year]-monetary_data["tud_tb_npv"][:last_year]), lw=1.0, c=color_tud_tb)
ax7c.plot(monetary_data_noisy["t"][:last_year], 1000*(monetary_data_noisy["tud_tbhb_noisy_speeds_npv"][:last_year]-monetary_data["tud_tbhb_npv"][:last_year]), lw=1.0, c=color_tud_tbhb)


ax7c.set_xlim(0, last_year)
ax7c.set_ylim(-40, 40)

ax7c.tick_params(axis="y", labelsize=tick_fs)
ax7c.tick_params(axis="x", labelsize=tick_fs)

ax7c.set_xlabel("Year", fontsize=axis_fs)
ax7c.set_ylabel("Change in NPV [m EUR]", fontsize=axis_fs)

ax7c_in = ax7c.inset_axes([0.62, 0.08, 0.33, 0.33])
for i in range(1, noise_samples+1):
    ax7c_in.plot(monetary_data_noisy["t"][:last_year], 1000*(monetary_data_noisy["dtu_ta_noisy_speeds_npv"][:last_year]-monetary_data["dtu_ta_npv"][:last_year]), lw=0.5, c=color_dtu_ta)
    ax7c_in.plot(monetary_data_noisy["t"][:last_year], 1000*(monetary_data_noisy[f"dtu_ta_noisy_speeds_{i}_npv"][:last_year]-monetary_data["dtu_ta_npv"][:last_year]), lw=0.5, c=color_dtu_ta, alpha=0.2)

"""print(f"TA noisy velocities: {total_deviation(1000*monetary_data_noisy["dtu_ta_noisy_speeds_npv"][:last_year], 1000*monetary_data["dtu_ta_npv"][:last_year], monetary_data_noisy["t"][:last_year]):3.0f}")
print(f"Greedy noisy velocities: {total_deviation(1000*monetary_data_noisy["dtu_greedy_noisy_speeds_npv"][:last_year], 1000*monetary_data["dtu_greedy_npv"][:last_year], monetary_data_noisy["t"][:last_year]):3.0f}")
print(f"Penalty noisy velocities: {total_deviation(1000*monetary_data_noisy["p_noisy_speeds_npv"][:last_year], 1000*monetary_data["tud_p_npv"][:last_year], monetary_data_noisy["t"][:last_year]):3.0f}")
print(f"TB noisy velocities: {total_deviation(1000*monetary_data_noisy["cs_noisy_speeds_npv"][:last_year], 1000*monetary_data["tud_tb_npv"][:last_year], monetary_data_noisy["t"][:last_year]):3.0f}")
print(f"TBHB noisy velocities: {total_deviation(1000*monetary_data_noisy["cshb_noisy_speeds_npv"][:last_year], 1000*monetary_data["tud_tbhb_npv"][:last_year], monetary_data_noisy["t"][:last_year]):3.0f}")"""

#ax7c_in.set_ylabel("NPV [b EUR]", fontsize=axis_fs-2, labelpad=0)
ax7c_in.set_xlim(0, last_year)
ax7c_in.set_ylim(-100, 25)
ax7c_in.xaxis.set_major_locator(ticker.MultipleLocator(10))
#ax7c_in.yaxis.set_major_locator(ticker.MultipleLocator(1))
ax7c_in.tick_params(axis="x", direction="in", labelsize=tick_fs-1, pad=1.5)
ax7c_in.tick_params(axis="y", direction="in", labelsize=tick_fs-1, pad=1.5)

fig7c.savefig(join(figures, f"FIG7c.{file_format}"))

plt.show()

# Methods
## Network

In [ ]:
total_length = sum([d["length"]/1000 for u, v, d in g.edges(data=True)])
sbpn_length = sum([d["length"]/1000 for u, v, d in g.edges(data=True) if d["highway"]=="residential"])
eschn_length = sum([d["length"]/1000 for u, v, d in g.edges(data=True) if d["ex_inf"]])
bschn_lenght = sum([d["length"]/1000 for u, v, d in g.edges(data=True) if d["seg_id"]!=-1])
uschn_lenght = sum([d["length"]/1000 for u, v, d in g.edges(data=True) if (d["seg_id"]!=-1 and d["Active_Basis"])])
nschn_lenght = sum([d["length"]/1000 for u, v, d in g.edges(data=True) if (d["seg_id"]!=-1 and not d["Active_Basis"])])

In [ ]:
print(f"Total Network Length: {total_length:1.3f} km")
print(f"Standard Bike Path Network Length: {sbpn_length:1.3f} km")
print(f"Standard Bike Path Network Fraction: {sbpn_length/total_length*100:5.2f} %")
print(f"Existing Cycle Superhighway Network Length: {eschn_length:1.3f} km")
print(f"Existing Cycle Superhighway Network Fraction: {eschn_length/total_length*100:5.2f} %")
print(f"Buildable Cycle Superhighway Network Length: {bschn_lenght:1.3f} km")
print(f"Buildable Cycle Superhighway Network Fraction: {bschn_lenght/total_length*100:5.2f} %")
print(f"Upgraded Cycle Superhighway Network Length: {uschn_lenght:1.3f} km")
print(f"Upgraded Cycle Superhighway Network Fraction: {uschn_lenght/total_length*100:5.2f} %")
print(f"New Cycle Superhighway Network Length: {nschn_lenght:1.3f} km")
print(f"New Cycle Superhighway Network Fraction: {nschn_lenght/total_length*100:5.2f} %")

In [ ]:
segment_cost_descriptor = (segment_cost[["CC per km", "MC per km"]]).describe() * 1000000

print("Construction Cost")
print(f"Min: {segment_cost_descriptor["CC per km"]["min"]:8.2f} EUR/km")
print(f"Max: {segment_cost_descriptor["CC per km"]["max"]:8.2f} EUR/km")
print(f"Average: {segment_cost_descriptor["CC per km"]["mean"]:8.2f} EUR/km")
print(f"Median: {segment_cost_descriptor["CC per km"]["50%"]:8.2f} EUR/km \n")

print("Maintenance Cost")
print(f"Min: {segment_cost_descriptor["MC per km"]["min"]:8.2f} EUR/km")
print(f"Max: {segment_cost_descriptor["MC per km"]["max"]:8.2f} EUR/km")
print(f"Average: {segment_cost_descriptor["MC per km"]["mean"]:8.2f} EUR/km")
print(f"Median: {segment_cost_descriptor["MC per km"]["50%"]:8.2f} EUR/km")

In [ ]:
with open(join(paths["input_folder"], tud_save, f"{tud_save}_demand.json"), "r") as f:
  demand = json.load(f)
od_pairs = {literal_eval(k) for k in demand.keys()}
od_points = {k_i for k in od_pairs for k_i in k}
unique_trips = {(literal_eval(k)[0], literal_eval(k)[1], int(v_k)) for k, v in demand.items() for v_k in v.keys()}

In [ ]:
print(f"Starting/End Points: {len(od_points)}")
print(f"Origin-Destination Pairs: {len(od_pairs)}")
print(f"Unique Trips: {len(unique_trips)}")

# Supplement
## Complete Overview

In [ ]:
overview_data = pd.DataFrame({"t": monetary_data["t"]})
for save in ["dtu_ta", "dtu_greedy", "tud_p", "tud_tb", "tud_tbhb"]:
    overview_data[f"{save}_TLB"] = [sum([segment_cost.loc[segment_cost["SegmentId"] == seg_id, "LengthGraph"].iloc[0] for seg_id in order_yearly[save][year]]) for year in monetary_data["t"]]
    overview_data[f"{save}_TLB_rel"] = [sum([segment_cost.loc[segment_cost["SegmentId"] == seg_id, "LengthGraph"].iloc[0] /total_segment_length for seg_id in order_yearly[save][year]]) for year in monetary_data["t"]]
    overview_data[f"{save}_CC"] = [sum([segment_cost.loc[segment_cost["SegmentId"] == seg_id, "ConstructionCosts"].iloc[0] for seg_id in order_yearly[save][year]]) for year in monetary_data["t"]]
    overview_data[f"{save}_CC_rel"] = [sum([segment_cost.loc[segment_cost["SegmentId"] == seg_id, "ConstructionCosts"].iloc[0] / total_segment_cost for seg_id in order_yearly[save][year]]) for year in monetary_data["t"]]
    overview_data[f"{save}_MC"] = [sum([segment_cost.loc[segment_cost["SegmentId"] == seg_id, "MaintenanceCosts"].iloc[0] for seg_id in order_yearly[save][year-1]]) for year in monetary_data["t"]]
    overview_data[f"{save}_NPV"] = monetary_data[f"{save}_npv"]
    overview_data[f"{save}_TB"] = monetary_data[f"{save}_tb"]
    overview_data[f"{save}_HB"] = monetary_data[f"{save}_hb"]

In [ ]:
fig_co, ax_co = plt.subplots(2, 3, figsize=(7.5, 4.5), dpi=dpi, sharex=True)
fig_co.tight_layout(pad=1.0, h_pad=0.5, w_pad=0.1)

ax_co_in_npv = ax_co[0, 0].inset_axes([0.62, 0.07, 0.35, 0.35])
ax_co_in_npv.axhline(0, color="#808080", linestyle="--", lw=0.5)
ax_co_in_tb = ax_co[0, 1].inset_axes([0.62, 0.07, 0.35, 0.35])
ax_co_in_tb.axhline(0, color="#808080", linestyle="--", lw=0.5)
ax_co_in_hb = ax_co[0, 2].inset_axes([0.62, 0.07, 0.35, 0.35])
ax_co_in_hb.axhline(0, color="#808080", linestyle="--", lw=0.5)
ax_co_in_cc = ax_co[1, 1].inset_axes([0.62, 0.07, 0.35, 0.35])
ax_co_in_tlb = ax_co[1, 0].inset_axes([0.62, 0.07, 0.35, 0.35])
for save in ["dtu_ta", "dtu_greedy", "tud_p", "tud_tb", "tud_tbhb"]:   
    ax_co[0, 0].plot(monetary_data["t"][:last_year], overview_data[f"{save}_NPV"][:last_year], "o", ms=2, c=color[f"color_{save}"])
    ax_co[0, 1].plot(monetary_data["t"][:last_year], overview_data[f"{save}_TB"][:last_year], "o", ms=2, c=color[f"color_{save}"])
    ax_co[0, 2].plot(monetary_data["t"][:last_year], overview_data[f"{save}_HB"][:last_year], "o", ms=2, c=color[f"color_{save}"])
    ax_co[1, 0].plot(monetary_data["t"][:last_year], overview_data[f"{save}_TLB"][:last_year], "o", ms=2, c=color[f"color_{save}"])
    ax_co_in_tlb.plot(monetary_data["t"][:last_year], overview_data[f"{save}_TLB_rel"][:last_year], lw=0.5, c=color[f"color_{save}"])
    ax_co[1, 1].plot(monetary_data["t"][:last_year], overview_data[f"{save}_CC"][:last_year], "o", ms=2, c=color[f"color_{save}"])
    ax_co_in_cc.plot(monetary_data["t"][:last_year], overview_data[f"{save}_CC_rel"][:last_year], lw=0.5, c=color[f"color_{save}"])
    ax_co[1, 2].plot(monetary_data["t"][:last_year], overview_data[f"{save}_MC"][:last_year], "o", ms=2, c=color[f"color_{save}"])
    if save != "dtu_ta":
        ax_co_in_npv.plot(monetary_data["t"][:last_year], 100*(overview_data[f"{save}_NPV"][:last_year]-overview_data["dtu_ta_NPV"][:last_year])/overview_data["dtu_ta_NPV"][:last_year], lw=0.5, c=color[f"color_{save}"])
        ax_co_in_tb.plot(monetary_data["t"][:last_year], 100*(overview_data[f"{save}_TB"][:last_year]-overview_data["dtu_ta_TB"][:last_year])/overview_data["dtu_ta_TB"][:last_year], lw=0.5, c=color[f"color_{save}"])
        ax_co_in_hb.plot(monetary_data["t"][:last_year], 100*(overview_data[f"{save}_HB"][:last_year]-overview_data["dtu_ta_HB"][:last_year])/overview_data["dtu_ta_HB"][:last_year], lw=0.5, c=color[f"color_{save}"])

ax_co[0, 0].text(0.02, 0.95, "a", fontsize=7, fontweight="bold", transform=ax_co[0, 0].transAxes)
ax_co[0, 1].text(0.02, 0.95, "b", fontsize=7, fontweight="bold", transform=ax_co[0, 1].transAxes)
ax_co[0, 2].text(0.02, 0.95, "c", fontsize=7, fontweight="bold", transform=ax_co[0, 2].transAxes)
ax_co[1, 0].text(0.02, 0.95, "d", fontsize=7, fontweight="bold", transform=ax_co[1, 0].transAxes)
ax_co[1, 1].text(0.02, 0.95, "e", fontsize=7, fontweight="bold", transform=ax_co[1, 1].transAxes)
ax_co[1, 2].text(0.02, 0.95, "f", fontsize=7, fontweight="bold", transform=ax_co[1, 2].transAxes)

plt.setp(ax_co, xlim=(0, last_year+1))
plt.setp(ax_co_in_npv, xlim=(0, last_year))
plt.setp(ax_co_in_tb, xlim=(0, last_year))
plt.setp(ax_co_in_hb, xlim=(0, last_year))
plt.setp(ax_co_in_tlb, xlim=(0, last_year), ylim=(0, 1))
plt.setp(ax_co_in_cc, xlim=(0, last_year), ylim=(0, 1))
ax_co[1, 0].set_ylim(0, 2000)
ax_co[1, 1].set_ylim(0, 250)
ax_co[1, 2].set_ylim(0, 20)

ax_co[1, 0].tick_params(axis="x", labelsize=tick_fs)
ax_co[1, 1].tick_params(axis="x", labelsize=tick_fs)
ax_co_in_npv.tick_params(axis="x", direction="in", labelsize=tick_fs-1, pad=1.5)
ax_co_in_tb.tick_params(axis="x", direction="in", labelsize=tick_fs-1, pad=1.5)
ax_co_in_hb.tick_params(axis="x", direction="in", labelsize=tick_fs-1, pad=1.5)
ax_co_in_tlb.tick_params(axis="x", direction="in", labelsize=tick_fs-1, pad=1.5)
ax_co_in_cc.tick_params(axis="x", direction="in", labelsize=tick_fs-1, pad=1.5)
ax_co[1, 2].tick_params(axis="x", labelsize=tick_fs)
ax_co[1, 0].xaxis.set_major_locator(ticker.MultipleLocator(10))
ax_co_in_tlb.xaxis.set_major_locator(ticker.MultipleLocator(10))
ax_co[1, 1].xaxis.set_major_locator(ticker.MultipleLocator(10))
ax_co_in_cc.xaxis.set_major_locator(ticker.MultipleLocator(10))
ax_co[1, 2].xaxis.set_major_locator(ticker.MultipleLocator(10))

ax_co[0, 0].tick_params(axis="y", labelsize=tick_fs)
ax_co_in_npv.tick_params(axis="y", direction="in", labelsize=tick_fs-1, pad=1.5)
ax_co[0, 1].tick_params(axis="y", labelsize=tick_fs)
ax_co_in_tb.tick_params(axis="y", direction="in", labelsize=tick_fs-1, pad=1.5)
ax_co[0, 2].tick_params(axis="y", labelsize=tick_fs)
ax_co_in_hb.tick_params(axis="y", direction="in", labelsize=tick_fs-1, pad=1.5)
ax_co[1, 0].tick_params(axis="y", labelsize=tick_fs)
ax_co_in_tlb.tick_params(axis="y", direction="in", labelsize=tick_fs-1, pad=1.5)
ax_co[1, 1].tick_params(axis="y", labelsize=tick_fs)
ax_co_in_cc.tick_params(axis="y", direction="in", labelsize=tick_fs-1, pad=1.5)
ax_co[1, 2].tick_params(axis="y", labelsize=tick_fs)
ax_co_in_tlb.yaxis.set_major_locator(ticker.MultipleLocator(0.5))
ax_co_in_cc.yaxis.set_major_locator(ticker.MultipleLocator(0.5))

ax_co[1, 0].set_xlabel("Year", fontsize=axis_fs)
ax_co[1, 1].set_xlabel("Year", fontsize=axis_fs)
ax_co[1, 2].set_xlabel("Year", fontsize=axis_fs)
ax_co[0, 0].set_ylabel("Net present value [b EUR]", fontsize=axis_fs)
ax_co_in_npv.set_ylabel(r"$\mathrm{\Delta}$ NPV [%]", fontsize=axis_fs-2, labelpad=0)
ax_co[0, 1].set_ylabel("Consumer surplus [b EUR]", fontsize=axis_fs)
ax_co_in_tb.set_ylabel(r"$\mathrm{\Delta}$ TB [%]", fontsize=axis_fs-2, labelpad=0)
ax_co[0, 2].set_ylabel("Health benefits [b EUR]", fontsize=axis_fs)
ax_co_in_hb.set_ylabel(r"$\mathrm{\Delta}$ HB [%]", fontsize=axis_fs-2, labelpad=0)
ax_co[1, 0].set_ylabel("Total length build [km]", fontsize=axis_fs)
ax_co_in_tlb.set_ylabel("Relative length build", fontsize=axis_fs-2, labelpad=0)
ax_co[1, 1].set_ylabel("Construction cost [m EUR]", fontsize=axis_fs)
ax_co_in_cc.set_ylabel("Relative CC", fontsize=axis_fs-2, labelpad=0)
ax_co[1, 2].set_ylabel("Maintenance cost [m EUR]", fontsize=axis_fs)


fig_co.savefig(join(figures, f"FIGS3_Performance_Overview.{file_format}"))   
plt.show()

## Planned Networks
### DTU TA

In [ ]:
fig_pn_dtu_ta, ax_bn_dtu_ta = plt.subplots(figsize=(5.6, 7.6), dpi=dpi)
fig_pn_dtu_ta.tight_layout()
ax_pn_dtu_ta = ax_bn_dtu_ta.twinx()

geom_streetnw.plot(ax=ax_bn_dtu_ta, color=color_street_nw, linewidth=0.1)
geom_existing.plot(ax=ax_bn_dtu_ta, color=color_exising_sch, linewidth=1)
geom_proposed.plot(ax=ax_pn_dtu_ta, color=color_upgradable_sch, linewidth=1)

ymin, ymax = min(ax_bn_dtu_ta.get_ylim()[0], ax_pn_dtu_ta.get_ylim()[0]), max(ax_bn_dtu_ta.get_ylim()[1], ax_pn_dtu_ta.get_ylim()[1])
ax_bn_dtu_ta.set_ylim(ymin, ymax)
ax_pn_dtu_ta.set_ylim(ymin, ymax)

ax_bn_dtu_ta.axis("off")
ax_pn_dtu_ta.axis("off")

fig_pn_dtu_ta.suptitle(f"Year 0")
fig_pn_dtu_ta.savefig(join(figures, "Planned Networks", "TA", f"Network_DTU_TA_Year_00.png"))

ax_pn_dtu_ta.cla()

for plot_year in range(1, 51):
    geom_proposed_plot = geom_proposed.copy()
    geom_proposed_plot.fillna({"Year_dtu_ta": 100}, inplace=True)
    geom_proposed_plot.loc[geom_proposed_plot["Year_dtu_ta"] <= plot_year, "plot_color"] = color_dtu_ta
    geom_proposed_plot.fillna({"plot_color": color_upgradable_sch}, inplace=True)
    geom_proposed_plot.loc[geom_proposed_plot["Year_dtu_ta"] <= plot_year, "plot_zorder"] = 1
    geom_proposed_plot.fillna({f"plot_zorder": 0}, inplace=True)

    geom_proposed_plot.sort_values(by=["plot_zorder"]).plot(ax=ax_pn_dtu_ta, color=geom_proposed_plot[f"plot_color"], linewidth=1)

    ax_pn_dtu_ta.set_ylim(ymin, ymax)
    ax_pn_dtu_ta.axis("off")
    
    fig_pn_dtu_ta.suptitle(f"Year {plot_year}")
    fig_pn_dtu_ta.savefig(join(figures, "Planned Networks", "TA", f"Network_DTU_TA_Year_{plot_year:02d}.png"))
    
    ax_pn_dtu_ta.cla()

plt.close()

### DTU Greedy

In [ ]:
fig_pn_dtu_greedy, ax_bn_dtu_greedy = plt.subplots(figsize=(5.6, 7.6), dpi=dpi)
fig_pn_dtu_greedy.tight_layout()
ax_pn_dtu_greedy = ax_bn_dtu_greedy.twinx()

geom_streetnw.plot(ax=ax_bn_dtu_greedy, color=color_street_nw, linewidth=0.1)
geom_existing.plot(ax=ax_bn_dtu_greedy, color=color_exising_sch, linewidth=1)
geom_proposed.plot(ax=ax_pn_dtu_greedy, color=color_upgradable_sch, linewidth=1)

ymin, ymax = min(ax_bn_dtu_ta.get_ylim()[0], ax_pn_dtu_greedy.get_ylim()[0]), max(ax_bn_dtu_greedy.get_ylim()[1], ax_pn_dtu_greedy.get_ylim()[1])
ax_bn_dtu_greedy.set_ylim(ymin, ymax)
ax_pn_dtu_greedy.set_ylim(ymin, ymax)

ax_bn_dtu_greedy.axis("off")
ax_pn_dtu_greedy.axis("off")

fig_pn_dtu_greedy.suptitle(f"Year 0")
fig_pn_dtu_greedy.savefig(join(figures, "Planned Networks", "LPGreedy", f"Network_DTU_LPGreedy_Year_00.png"))

ax_pn_dtu_greedy.cla()

for plot_year in range(1, 51):
    geom_proposed_plot = geom_proposed.copy()
    geom_proposed_plot.loc[geom_proposed_plot["Year_dtu_greedy"] <= plot_year, "plot_color"] = color_dtu_greedy
    geom_proposed_plot.fillna({"plot_color": color_upgradable_sch}, inplace=True)
    geom_proposed_plot.loc[geom_proposed_plot["Year_dtu_greedy"] <= plot_year, "plot_zorder"] = 1
    geom_proposed_plot.fillna({f"plot_zorder": 0}, inplace=True)

    geom_proposed_plot.sort_values(by=["plot_zorder"]).plot(ax=ax_pn_dtu_greedy, color=geom_proposed_plot["plot_color"], linewidth=1)

    ax_pn_dtu_greedy.set_ylim(ymin, ymax)
    ax_pn_dtu_greedy.axis("off")
    
    fig_pn_dtu_greedy.suptitle(f"Year {plot_year}")
    fig_pn_dtu_greedy.savefig(join(figures, "Planned Networks", "LPGreedy", f"Network_DTU_LPGreedy_Year_{plot_year:02d}.png"))
    
    ax_pn_dtu_greedy.cla()

plt.close()

### TUD TBHB

In [ ]:
fig_pn_tbhb, ax_bn_tbhb = plt.subplots(figsize=(5.6, 7.6), dpi=dpi)
fig_pn_tbhb.tight_layout()
ax_pn_tbhb = ax_bn_tbhb.twinx()

geom_streetnw.plot(ax=ax_bn_tbhb, color=color_street_nw, linewidth=0.1)
geom_existing.plot(ax=ax_bn_tbhb, color=color_exising_sch, linewidth=1)
geom_proposed.plot(ax=ax_pn_tbhb, color=color_upgradable_sch, linewidth=1)

ymin, ymax = min(ax_bn_tbhb.get_ylim()[0], ax_pn_tbhb.get_ylim()[0]), max(ax_bn_tbhb.get_ylim()[1], ax_pn_tbhb.get_ylim()[1])
ax_bn_tbhb.set_ylim(ymin, ymax)
ax_pn_tbhb.set_ylim(ymin, ymax)

ax_bn_tbhb.axis("off")
ax_pn_tbhb.axis("off")

fig_pn_tbhb.suptitle(f"Year 0")
fig_pn_tbhb.savefig(join(figures, "Planned Networks", "TBHB", f"Network_TUD_TBHB_Year_00.png"))

ax_pn_tbhb.cla()

for plot_year in range(1, 51):
    geom_proposed_plot = geom_proposed.copy()
    geom_proposed_plot.loc[geom_proposed_plot["Year_tud_tbhb"] <= plot_year, "plot_color"] = color_tud_tbhb
    geom_proposed_plot.fillna({"plot_color": color_upgradable_sch}, inplace=True)
    geom_proposed_plot.loc[geom_proposed_plot["Year_tud_tbhb"] <= plot_year, "plot_zorder"] = 1
    geom_proposed_plot.fillna({"plot_zorder": 0}, inplace=True)

    geom_proposed_plot.sort_values(by=["plot_zorder"]).plot(ax=ax_pn_tbhb, color=geom_proposed_plot["plot_color"], linewidth=1)

    ax_pn_tbhb.set_ylim(ymin, ymax)
    ax_pn_tbhb.axis("off")
    
    fig_pn_tbhb.suptitle(f"Year {plot_year}")
    fig_pn_tbhb.savefig(join(figures, "Planned Networks", "TBHB", f"Network_TUD_TBHB_Year_{plot_year:02d}.png"))
    
    ax_pn_tbhb.cla()

plt.close()

### TUD TB

In [ ]:
fig_pn_tb, ax_bn_tb = plt.subplots(figsize=(5.6, 7.6), dpi=dpi)
fig_pn_tb.tight_layout()
ax_pn_tb = ax_bn_tb.twinx()

geom_streetnw.plot(ax=ax_bn_tb, color=color_street_nw, linewidth=0.1)
geom_existing.plot(ax=ax_bn_tb, color=color_exising_sch, linewidth=1)
geom_proposed.plot(ax=ax_pn_tb, color=color_upgradable_sch, linewidth=1)

ymin, ymax = min(ax_bn_tb.get_ylim()[0], ax_pn_tb.get_ylim()[0]), max(ax_bn_tb.get_ylim()[1], ax_pn_tb.get_ylim()[1])
ax_bn_tb.set_ylim(ymin, ymax)
ax_pn_tb.set_ylim(ymin, ymax)

ax_bn_tb.axis("off")
ax_pn_tb.axis("off")

fig_pn_tb.suptitle(f"Year 0")
fig_pn_tb.savefig(join(figures, "Planned Networks", "TB", f"Network_TUD_TB_Year_00.png"))

ax_pn_tb.cla()

for plot_year in range(1, 51):
    geom_proposed_plot = geom_proposed.copy()
    geom_proposed_plot.loc[geom_proposed_plot["Year_tud_tb"] <= plot_year, "plot_color"] = color_tud_tb
    geom_proposed_plot.fillna({"plot_color": color_upgradable_sch}, inplace=True)
    geom_proposed_plot.loc[geom_proposed_plot["Year_tud_tb"] <= plot_year, "plot_zorder"] = 1
    geom_proposed_plot.fillna({"plot_zorder": 0}, inplace=True)

    geom_proposed_plot.sort_values(by=["plot_zorder"]).plot(ax=ax_pn_tb, color=geom_proposed_plot["plot_color"], linewidth=1)

    ax_pn_tb.set_ylim(ymin, ymax)
    ax_pn_tb.axis("off")
    
    fig_pn_tb.suptitle(f"Year {plot_year}")
    fig_pn_tb.savefig(join(figures, "Planned Networks", "TB", f"Network_TUD_TB_Year_{plot_year:02d}.png"))
    
    ax_pn_tb.cla()

plt.close()

### TUD Penalty

In [ ]:
fig_pn_p, ax_bn_p = plt.subplots(figsize=(5.6, 7.6), dpi=dpi)
fig_pn_p.tight_layout()
ax_pn_p = ax_bn_p.twinx()

geom_streetnw.plot(ax=ax_bn_p, color=color_street_nw, linewidth=0.1)
geom_existing.plot(ax=ax_bn_p, color=color_exising_sch, linewidth=1)
geom_proposed.plot(ax=ax_pn_p, color=color_upgradable_sch, linewidth=1)

ymin, ymax = min(ax_bn_p.get_ylim()[0], ax_pn_p.get_ylim()[0]), max(ax_bn_p.get_ylim()[1], ax_pn_p.get_ylim()[1])
ax_bn_p.set_ylim(ymin, ymax)
ax_pn_p.set_ylim(ymin, ymax)

ax_bn_p.axis("off")
ax_pn_p.axis("off")

fig_pn_p.suptitle(f"Year 0")
fig_pn_p.savefig(join(figures, "Planned Networks", "Penalty", f"Network_TUD_Penalty_Year_00.png"))

ax_pn_p.cla()

for plot_year in range(1, 51):
    geom_proposed_plot = geom_proposed.copy()
    geom_proposed_plot.loc[geom_proposed_plot["Year_tud_p"] <= plot_year, "plot_color"] = color_tud_p
    geom_proposed_plot.fillna({"plot_color": color_upgradable_sch}, inplace=True)
    geom_proposed_plot.loc[geom_proposed["Year_tud_p"] <= plot_year, "plot_zorder"] = 1
    geom_proposed_plot.fillna({"plot_zorder": 0}, inplace=True)

    geom_proposed_plot.sort_values(by=["plot_zorder"]).plot(ax=ax_pn_p, color=geom_proposed_plot["plot_color"], linewidth=1)

    ax_pn_p.set_ylim(ymin, ymax)
    ax_pn_p.axis("off")
    
    fig_pn_p.suptitle(f"Year {plot_year}")
    fig_pn_p.savefig(join(figures, "Planned Networks", "Penalty", f"Network_TUD_Penalty_Year_{plot_year:02d}.png"))
    
    ax_pn_p.cla()

plt.close()

## Robustness

In [ ]:
order_segment_noisy = pd.DataFrame({"segment": sorted(tud_p_order)})
order_segment_noisy["tud_p_base"] = [order_segment["tud_p"].to_list().index(seg_id)+1 for seg_id in order_segment_noisy["segment"]]
order_segment_noisy["tud_tb_base"] = [order_segment["tud_tb"].to_list().index(seg_id)+1 for seg_id in order_segment_noisy["segment"]]
order_segment_noisy["tud_tbhb_base"] = [order_segment["tud_tbhb"].to_list().index(seg_id)+1 for seg_id in order_segment_noisy["segment"]]
order_segment_noisy["dtu_greedy_base"] = [order_segment["dtu_greedy"].to_list().index(seg_id)+1 for seg_id in order_segment_noisy["segment"]]
order_segment_noisy["dtu_ta_base"] = order_segment_noisy["segment"].map({seg: idx+1 for idx, seg in enumerate(dtu_ta_order)})

order_yearly_noisy = pd.DataFrame({"year": all_years})
utility_noisy = pd.DataFrame()
ba_noisy_segment = pd.DataFrame()
ba_noisy_year = pd.DataFrame({"t": all_years})
for noisy in ["noisy_costs", "noisy_demand", "noisy_speeds"]:
    for i in range(1, noise_samples+1):
        tud_save_noisy = f"{tud_save}_{noisy}_{i}"
        dtu_save_noisy = f"{dtu_save}_{noisy}_{i}"
    
        with open(join(paths["output_folder"], tud_save_noisy, f"{tud_save_noisy}_order_Penalty.json"), "r") as f:
          tud_order_noisy_p = json.load(f)
        with open(join(paths["output_folder"], tud_save_noisy, f"{tud_save_noisy}_order_TB.json"), "r") as f:
          tud_order_noisy_tb = json.load(f)
        with open(join(paths["output_folder"], tud_save_noisy, f"{tud_save_noisy}_order_TBHB.json"), "r") as f:
          tud_order_noisy_tbhb = json.load(f)
        with open(join(paths["output_folder"], dtu_save_noisy, f"{dtu_save_noisy}_order_LPGreedy.json"), "r") as f:
          dtu_greedy_order_noisy = json.load(f)
        with open(join(paths["output_folder"], dtu_save_noisy, f"{dtu_save_noisy}_order_TA.json"), "r") as f:
          dtu_ta_order_noisy = json.load(f)
    
        order_segment_noisy[f"tud_p_{noisy}_{i}"] = [tud_order_noisy_p.index(seg_id)+1 for seg_id in order_segment_noisy["segment"]]
        order_segment_noisy[f"tud_tb_{noisy}_{i}"] = [tud_order_noisy_tb.index(seg_id)+1 for seg_id in order_segment_noisy["segment"]]
        order_segment_noisy[f"tud_tbhb_{noisy}_{i}"] = [tud_order_noisy_tbhb.index(seg_id)+1 for seg_id in order_segment_noisy["segment"]]
        order_segment_noisy[f"dtu_greedy_{noisy}_{i}"] = [dtu_greedy_order_noisy.index(seg_id)+1 for seg_id in order_segment_noisy["segment"]]
        order_segment_noisy[f"dtu_ta_{noisy}_{i}"] = order_segment_noisy["segment"].map({seg: idx+1 for idx, seg in enumerate(dtu_ta_order_noisy)})
        
        tud_order_yearly_noisy_p, tud_order_yearly_noisy_tb, tud_order_yearly_noisy_tbhb, dtu_greedy_order_yearly_noisy, dtu_ta_order_yearly_noisy = load_yearly_ordering(tud_save_noisy, dtu_save_noisy)
        order_yearly_noisy[f"tud_p_{noisy}_{i}"] = dict(sorted(tud_order_yearly_noisy_p.items())).values()
        order_yearly_noisy[f"tud_tb_{noisy}_{i}"] = dict(sorted(tud_order_yearly_noisy_tb.items())).values()
        order_yearly_noisy[f"tud_tbhb_{noisy}_{i}"] = dict(sorted(tud_order_yearly_noisy_tbhb.items())).values()
        order_yearly_noisy[f"dtu_greedy_{noisy}_{i}"] = dict(sorted(dtu_greedy_order_yearly_noisy.items())).values()
        order_yearly_noisy[f"dtu_ta_{noisy}_{i}"] = dict(sorted(dtu_ta_order_yearly_noisy.items())).values()
        
        with open(join(paths["output_folder"], tud_save_noisy, f"{tud_save_noisy}_utility_Penalty_eval.json"), "r") as f:
          tud_p_utility_data_noisy = json.load(f)
        with open(join(paths["output_folder"], tud_save_noisy, f"{tud_save_noisy}_utility_TB_eval.json"), "r") as f:
          tud_tb_utility_data_noisy = json.load(f)
        with open(join(paths["output_folder"], tud_save_noisy, f"{tud_save_noisy}_utility_TBHB_eval.json"), "r") as f:
          tud_tbhb_utility_data_noisy = json.load(f)
        with open(join(paths["output_folder"], dtu_save_noisy, f"{dtu_save_noisy}_utility_LPGreedy_eval.json"), "r") as f:
          dtu_greedy_utility_data_noisy = json.load(f)
        with open(join(paths["output_folder"], dtu_save_noisy, f"{dtu_save_noisy}_utility_TA_eval.json"), "r") as f:
          dtu_ta_utility_data_noisy = json.load(f)
        
        tud_p_utility_noisy_raw = pd.DataFrame(list(reversed(tud_p_utility_data_noisy)))
        utility_noisy[f"tud_p_{noisy}_{i}"] = tud_p_utility_noisy_raw.replace(to_replace=0.0, value=np.nan).dropna()[0].to_list()
        tud_tb_utility_noisy_raw = pd.DataFrame(list(reversed(tud_tb_utility_data_noisy)))
        utility_noisy[f"tud_tb_{noisy}_{i}"] = tud_tb_utility_noisy_raw.replace(to_replace=0.0, value=np.nan).dropna()[0].to_list()
        tud_tbhb_utility_noisy_raw = pd.DataFrame(list(reversed(tud_tbhb_utility_data_noisy)))
        utility_noisy[f"tud_tbhb_{noisy}_{i}"] = tud_tbhb_utility_noisy_raw.replace(to_replace=0.0, value=np.nan).dropna()[0].to_list()
        dtu_greedy_utility_noisy_raw = pd.DataFrame(list(reversed(dtu_greedy_utility_data_noisy)))
        utility_noisy[f"dtu_greedy_{noisy}_{i}"] = dtu_greedy_utility_noisy_raw.replace(to_replace=0.0, value=np.nan).dropna()[0].to_list()
        dtu_ta_utility_noisy_raw = pd.DataFrame(list(reversed(dtu_ta_utility_data_noisy)))
        dtu_ta_utility_noisy_raw = dtu_ta_utility_noisy_raw.replace(to_replace=0.0, value=np.nan).dropna()[0].to_list()
        dtu_ta_utility_noisy_raw.extend((len(dtu_greedy_order_noisy)-len(dtu_ta_order_noisy))*[dtu_ta_utility_noisy_raw[-1]])
        utility_noisy[f"dtu_ta_{noisy}_{i}"] = dtu_ta_utility_noisy_raw
    
        max_utility_noisy_tud_p = utility_noisy[f"tud_p_{noisy}_{i}"].max()
        max_utility_noisy_tud_tb = utility_noisy[f"tud_tb_{noisy}_{i}"].max()
        max_utility_noisy_tud_tbhb = utility_noisy[f"tud_tbhb_{noisy}_{i}"].max()
        max_utility_noisy_dtu_greedy = utility_noisy[f"dtu_greedy_{noisy}_{i}"].max()
        max_utility_noisy_dtu_ta = utility_noisy[f"dtu_ta_{noisy}_{i}"].max()
        max_utility = utility_noisy[[f"tud_p_{noisy}_{i}", f"tud_tb_{noisy}_{i}", f"tud_tbhb_{noisy}_{i}", f"dtu_greedy_{noisy}_{i}", f"dtu_ta_{noisy}_{i}"]].max().max()
        min_utility_noisy_tud_p = utility_noisy[f"tud_p_{noisy}_{i}"].min()
        min_utility_noisy_tud_tb = utility_noisy[f"tud_tb_{noisy}_{i}"].min()
        min_utility_noisy_tud_tbhb = utility_noisy[f"tud_tbhb_{noisy}_{i}"].min()
        min_utility_noisy_dtu_greedy = utility_noisy[f"dtu_greedy_{noisy}_{i}"].min()
        min_utility_noisy_dtu_ta = utility_noisy[f"dtu_ta_{noisy}_{i}"].min()
        min_utility = utility_noisy[[f"tud_p_{noisy}_{i}", f"tud_tb_{noisy}_{i}", f"tud_tbhb_{noisy}_{i}", f"dtu_greedy_{noisy}_{i}", f"dtu_ta_{noisy}_{i}"]].min().min()

    
        ba_noisy_segment[f"tud_p_{noisy}_{i}"] = [(min_utility - i) / (min_utility - max_utility) for i in utility_noisy[f"tud_p_{noisy}_{i}"]]
        ba_noisy_segment[f"tud_tb_{noisy}_{i}"] = [(min_utility - i) / (min_utility - max_utility) for i in utility_noisy[f"tud_tb_{noisy}_{i}"]]
        ba_noisy_segment[f"tud_tbhb_{noisy}_{i}"] = [(min_utility - i) / (min_utility - max_utility) for i in utility_noisy[f"tud_tbhb_{noisy}_{i}"]]
        ba_noisy_segment[f"dtu_greedy_{noisy}_{i}"] = [(min_utility - i) / (min_utility - max_utility) for i in utility_noisy[f"dtu_greedy_{noisy}_{i}"]]
        ba_noisy_segment[f"dtu_ta_{noisy}_{i}"] = [(min_utility - i) / (min_utility - max_utility) for i in utility_noisy[f"dtu_ta_{noisy}_{i}"]]
        
        ba_noisy_year[f"tud_p_{noisy}_{i}"] = [ba_noisy_segment[f"tud_p_{noisy}_{i}"].to_list()[len(order_yearly_noisy.iloc[year][f"tud_p_{noisy}_{i}"])] for year in ba_noisy_year["t"]]
        ba_noisy_year[f"tud_tb_{noisy}_{i}"] = [ba_noisy_segment[f"tud_tb_{noisy}_{i}"].to_list()[len(order_yearly_noisy.iloc[year][f"tud_tb_{noisy}_{i}"])] for year in ba_noisy_year["t"]]
        ba_noisy_year[f"tud_tbhb_{noisy}_{i}"] = [ba_noisy_segment[f"tud_tbhb_{noisy}_{i}"].to_list()[len(order_yearly_noisy.iloc[year][f"tud_tbhb_{noisy}_{i}"])] for year in ba_noisy_year["t"]]
        ba_noisy_year[f"dtu_greedy_{noisy}_{i}"] = [ba_noisy_segment[f"dtu_greedy_{noisy}_{i}"].to_list()[len(order_yearly_noisy.iloc[year][f"dtu_greedy_{noisy}_{i}"])] for year in ba_noisy_year["t"]]
        ba_noisy_year[f"dtu_ta_{noisy}_{i}"] = [ba_noisy_segment[f"dtu_ta_{noisy}_{i}"].to_list()[len(order_yearly_noisy.iloc[year][f"dtu_ta_{noisy}_{i}"])] for year in ba_noisy_year["t"]]
        
    ba_noisy_segment[f"tud_p_{noisy}"] = ba_noisy_segment[[f"tud_p_{noisy}_{i}" for i in range(1, noise_samples+1)]].mean(axis=1)
    ba_noisy_segment[f"tud_tb_{noisy}"] = ba_noisy_segment[[f"tud_tb_{noisy}_{i}" for i in range(1, noise_samples+1)]].mean(axis=1)
    ba_noisy_segment[f"tud_tbhb_{noisy}"] = ba_noisy_segment[[f"tud_tbhb_{noisy}_{i}" for i in range(1, noise_samples+1)]].mean(axis=1)
    ba_noisy_segment[f"dtu_greedy_{noisy}"] = ba_noisy_segment[[f"dtu_greedy_{noisy}_{i}" for i in range(1, noise_samples+1)]].mean(axis=1)
    ba_noisy_segment[f"dtu_ta_{noisy}"] = ba_noisy_segment[[f"dtu_ta_{noisy}_{i}" for i in range(1, noise_samples+1)]].mean(axis=1)
    
    ba_noisy_year[f"tud_p_{noisy}"] = ba_noisy_year[[f"tud_p_{noisy}_{i}" for i in range(1, noise_samples+1)]].mean(axis=1)
    ba_noisy_year[f"tud_tb_{noisy}"] = ba_noisy_year[[f"tud_tb_{noisy}_{i}" for i in range(1, noise_samples+1)]].mean(axis=1)
    ba_noisy_year[f"tud_tbhb_{noisy}"] = ba_noisy_year[[f"tud_tbhb_{noisy}_{i}" for i in range(1, noise_samples+1)]].mean(axis=1)
    ba_noisy_year[f"dtu_greedy_{noisy}"] = ba_noisy_year[[f"dtu_greedy_{noisy}_{i}" for i in range(1, noise_samples+1)]].mean(axis=1)
    ba_noisy_year[f"dtu_ta_{noisy}"] = ba_noisy_year[[f"dtu_ta_{noisy}_{i}" for i in range(1, noise_samples+1)]].mean(axis=1)

In [ ]:
fig_rob_metrics, ax_rob_metrics = plt.subplots(4, 3, figsize=(7.5, 9), dpi=dpi, sharey="row", sharex="row")
fig_rob_metrics.tight_layout(pad=1.0, h_pad=0.5, w_pad=0.1)

subplot_numbering = 0

for clm, noisy in enumerate(["noisy_demand", "noisy_costs", "noisy_speeds"]):
    ax_rob_metrics[0, clm].axhline(0, color="#808080", linestyle="--", lw=0.5)
    ax_rob_metrics[0, clm].plot(monetary_data_noisy["t"][:last_year], 1000*(monetary_data_noisy[f"dtu_ta_{noisy}_tb"][:last_year]-monetary_data["dtu_ta_tb"][:last_year]), lw=1, c=color_dtu_ta)
    ax_rob_metrics[0, clm].plot(monetary_data_noisy["t"][:last_year], 1000*(monetary_data_noisy[f"dtu_greedy_{noisy}_tb"][:last_year]-monetary_data["dtu_greedy_tb"][:last_year]), lw=1, c=color_dtu_greedy)
    ax_rob_metrics[0, clm].plot(monetary_data_noisy["t"][:last_year], 1000*(monetary_data_noisy[f"tud_p_{noisy}_tb"][:last_year]-monetary_data["tud_p_tb"][:last_year]), lw=1, c=color_tud_p)
    ax_rob_metrics[0, clm].plot(monetary_data_noisy["t"][:last_year], 1000*(monetary_data_noisy[f"tud_tb_{noisy}_tb"][:last_year]-monetary_data["tud_tb_tb"][:last_year]), lw=1, c=color_tud_tb)
    ax_rob_metrics[0, clm].plot(monetary_data_noisy["t"][:last_year], 1000*(monetary_data_noisy[f"tud_tbhb_{noisy}_tb"][:last_year]-monetary_data["tud_tbhb_tb"][:last_year]), lw=1, c=color_tud_tbhb)
    
    ax_rob_metrics[0, clm].set_xlim(0, last_year)
    ax_rob_metrics[0, clm].set_ylim(-13, 4)
    ax_rob_metrics[0, clm].xaxis.set_major_locator(ticker.MultipleLocator(10))
    ax_rob_metrics[0, clm].yaxis.set_major_locator(ticker.MultipleLocator(4))
    ax_rob_metrics[0, clm].tick_params(axis="y", labelsize=tick_fs, pad=1.5)
    ax_rob_metrics[0, clm].tick_params(axis="x", labelsize=tick_fs, pad=1.5)
    ax_rob_metrics[0, clm].set_xlabel("Year", fontsize=axis_fs)
    ax_rob_metrics[0, clm].text(0.02, 0.95, chr(ord("a") + subplot_numbering), fontsize=7, fontweight="bold", transform=ax_rob_metrics[0, clm].transAxes)
    
    ax_rob_metrics[1, clm].axhline(0, color="#808080", linestyle="--", lw=0.5)
    ax_rob_metrics[1, clm].plot(monetary_data_noisy["t"][:last_year], 1000*(monetary_data_noisy[f"dtu_ta_{noisy}_hb"][:last_year]-monetary_data["dtu_ta_hb"][:last_year]), lw=1, c=color_dtu_ta)
    ax_rob_metrics[1, clm].plot(monetary_data_noisy["t"][:last_year], 1000*(monetary_data_noisy[f"dtu_greedy_{noisy}_hb"][:last_year]-monetary_data["dtu_greedy_hb"][:last_year]), lw=1, c=color_dtu_greedy)
    ax_rob_metrics[1, clm].plot(monetary_data_noisy["t"][:last_year], 1000*(monetary_data_noisy[f"tud_p_{noisy}_hb"][:last_year]-monetary_data["tud_p_hb"][:last_year]), lw=1, c=color_tud_p)
    ax_rob_metrics[1, clm].plot(monetary_data_noisy["t"][:last_year], 1000*(monetary_data_noisy[f"tud_tb_{noisy}_hb"][:last_year]-monetary_data["tud_tb_hb"][:last_year]), lw=1, c=color_tud_tb)
    ax_rob_metrics[1, clm].plot(monetary_data_noisy["t"][:last_year], 1000*(monetary_data_noisy[f"tud_tbhb_{noisy}_hb"][:last_year]-monetary_data["tud_tbhb_hb"][:last_year]), lw=1, c=color_tud_tbhb)

    ax_rob_metrics[1, clm].set_xlim(0, last_year)
    ax_rob_metrics[1, clm].set_ylim(-120, 30)
    ax_rob_metrics[1, clm].xaxis.set_major_locator(ticker.MultipleLocator(10))
    ax_rob_metrics[1, clm].yaxis.set_major_locator(ticker.MultipleLocator(20))
    ax_rob_metrics[1, clm].tick_params(axis="y", labelsize=tick_fs, pad=1.5)
    ax_rob_metrics[1, clm].tick_params(axis="x",  labelsize=tick_fs, pad=1.5)
    ax_rob_metrics[1, clm].set_xlabel("Year", fontsize=axis_fs)
    ax_rob_metrics[1, clm].text(0.02, 0.95, chr(ord("a") + subplot_numbering + 3), fontsize=7, fontweight="bold", transform=ax_rob_metrics[1, clm].transAxes)
    
    ax_rob_metrics[2, clm].axhline(0, color="#808080", linestyle="--", lw=0.5)
    ax_rob_metrics[2, clm].plot(ba_noisy_year["t"][:last_year+1], ba_noisy_year[f"dtu_ta_{noisy}"][:last_year+1]-ba_year["dtu_ta"][:last_year+1], lw=1, c=color_dtu_ta)
    ax_rob_metrics[2, clm].plot(ba_noisy_year["t"][:last_year+1], ba_noisy_year[f"dtu_greedy_{noisy}"][:last_year+1]-ba_year["dtu_greedy"][:last_year+1], lw=1, c=color_dtu_greedy)
    ax_rob_metrics[2, clm].plot(ba_noisy_year["t"][:last_year+1], ba_noisy_year[f"tud_p_{noisy}"][:last_year+1]-ba_year["tud_p"][:last_year+1], lw=1, c=color_tud_p)
    ax_rob_metrics[2, clm].plot(ba_noisy_year["t"][:last_year+1], ba_noisy_year[f"tud_tb_{noisy}"][:last_year+1]-ba_year["tud_tb"][:last_year+1], lw=1, c=color_tud_tb)
    ax_rob_metrics[2, clm].plot(ba_noisy_year["t"][:last_year+1], ba_noisy_year[f"tud_tbhb_{noisy}"][:last_year+1]-ba_year["tud_tbhb"][:last_year+1], lw=1, c=color_tud_tbhb)
    
    ax_rob_metrics[2, clm].set_xlim(0, last_year)
    ax_rob_metrics[2, clm].set_ylim(-0.07, 0.07)
    ax_rob_metrics[2, clm].xaxis.set_major_locator(ticker.MultipleLocator(10))
    ax_rob_metrics[2, clm].yaxis.set_major_locator(ticker.MultipleLocator(0.02))
    ax_rob_metrics[2, clm].tick_params(axis="y", labelsize=tick_fs, pad=1.5)
    ax_rob_metrics[2, clm].tick_params(axis="x", labelsize=tick_fs, pad=1.5)
    ax_rob_metrics[2, clm].set_xlabel("Year", fontsize=axis_fs)
    ax_rob_metrics[2, clm].text(0.02, 0.95, chr(ord("a") + subplot_numbering + 6), fontsize=7, fontweight="bold", transform=ax_rob_metrics[2, clm].transAxes)
    
    ax_rob_metrics[3, clm].axhline(0, color="#808080", linestyle="--", lw=0.5)
    ax_rob_metrics[3, clm].plot(segments_added, ba_noisy_segment[f"dtu_ta_{noisy}"]-ba_segment["dtu_ta"], lw=1, c=color_dtu_ta)
    ax_rob_metrics[3, clm].plot(segments_added, ba_noisy_segment[f"dtu_greedy_{noisy}"]-ba_segment["dtu_greedy"], lw=1, c=color_dtu_greedy)
    ax_rob_metrics[3, clm].plot(segments_added, ba_noisy_segment[f"tud_p_{noisy}"]-ba_segment["tud_p"], lw=1, c=color_tud_p)
    ax_rob_metrics[3, clm].plot(segments_added, ba_noisy_segment[f"tud_tb_{noisy}"]-ba_segment["tud_tb"], lw=1, c=color_tud_tb)
    ax_rob_metrics[3, clm].plot(segments_added, ba_noisy_segment[f"tud_tbhb_{noisy}"]-ba_segment["tud_tbhb"], lw=1, c=color_tud_tbhb)

    ax_rob_metrics[3, clm].set_xlim(0, 202)
    ax_rob_metrics[3, clm].set_ylim(-0.07, 0.07)
    ax_rob_metrics[3, clm].xaxis.set_major_locator(ticker.MultipleLocator(50))
    ax_rob_metrics[3, clm].yaxis.set_major_locator(ticker.MultipleLocator(0.02))
    ax_rob_metrics[3, clm].tick_params(axis="y", labelsize=tick_fs, pad=1.5)
    ax_rob_metrics[3, clm].tick_params(axis="x",  labelsize=tick_fs, pad=1.5)
    ax_rob_metrics[3, clm].set_xlabel("Number of segments added", fontsize=axis_fs)
    ax_rob_metrics[3, clm].text(0.02, 0.95, chr(ord("a") + subplot_numbering + 9), fontsize=7, fontweight="bold", transform=ax_rob_metrics[3, clm].transAxes)
    
    subplot_numbering += 1

ax_rob_metrics[0, 0].set_ylabel("Change in TB [m EUR]", fontsize=axis_fs)
ax_rob_metrics[1, 0].set_ylabel("Change in HB [m EUR]", fontsize=axis_fs)
ax_rob_metrics[2, 0].set_ylabel("Change in bikeability", fontsize=axis_fs)
ax_rob_metrics[3, 0].set_ylabel("Change in bikeability", fontsize=axis_fs)

ax_rob_metrics[0, 0].set_title("Noisy demand", fontsize=axis_fs)
ax_rob_metrics[0, 1].set_title("Noisy costs", fontsize=axis_fs)
ax_rob_metrics[0, 2].set_title("Noisy velocities", fontsize=axis_fs)

fig_rob_metrics.savefig(join(figures, f"FIGS4_Robustness_Metrics_Mean.{file_format}"))   
plt.show()

In [ ]:
fig_rob_rank, ax_rob_rank = plt.subplots(5, 3, figsize=(7.5, 10), dpi=dpi, sharex=True, sharey=True)
fig_rob_rank.tight_layout(pad=1.0, h_pad=0.5, w_pad=0.1)

outlier_diff_rank = 20

subplot_numbering = 0
for clm, noisy in enumerate(["noisy_demand", "noisy_costs", "noisy_speeds"]):
    tud_p_x, tud_p_y = [], []
    tud_tb_x, tud_tb_y = [], []
    tud_tbhb_x, tud_tbhb_y = [], []
    dtu_ta_x, dtu_ta_y = [], []
    dtu_greedy_x, dtu_greedy_y = [], []
    
    ax_rob_rank[0, clm].plot(order_segment_noisy["segment"], order_segment_noisy["segment"], "--", c="#808080", alpha=0.5, zorder=2)
    ax_rob_rank[0, clm].plot(order_segment_noisy["segment"], order_segment_noisy["segment"]+outlier_diff_rank, "--", c="#808080", alpha=0.15, lw=0.5, zorder=2)
    ax_rob_rank[0, clm].plot(order_segment_noisy["segment"], order_segment_noisy["segment"]-outlier_diff_rank, "--", c="#808080", alpha=0.15, lw=0.5, zorder=2)
    ax_rob_rank[1, clm].plot(order_segment_noisy["segment"], order_segment_noisy["segment"], "--", c="#808080", alpha=0.5, zorder=2)
    ax_rob_rank[1, clm].plot(order_segment_noisy["segment"], order_segment_noisy["segment"]+outlier_diff_rank, "--", c="#808080", alpha=0.15, lw=0.5, zorder=2)
    ax_rob_rank[1, clm].plot(order_segment_noisy["segment"], order_segment_noisy["segment"]-outlier_diff_rank, "--", c="#808080", alpha=0.15, lw=0.5, zorder=2)
    ax_rob_rank[2, clm].plot(order_segment_noisy["segment"], order_segment_noisy["segment"], "--", c="#808080", alpha=0.5, zorder=2)
    ax_rob_rank[2, clm].plot(order_segment_noisy["segment"], order_segment_noisy["segment"]+outlier_diff_rank, "--", c="#808080", alpha=0.15, lw=0.5, zorder=2)
    ax_rob_rank[2, clm].plot(order_segment_noisy["segment"], order_segment_noisy["segment"]-outlier_diff_rank, "--", c="#808080", alpha=0.15, lw=0.5, zorder=2)
    ax_rob_rank[3, clm].plot(order_segment_noisy["segment"], order_segment_noisy["segment"], "--", c="#808080", alpha=0.5, zorder=2)
    ax_rob_rank[3, clm].plot(order_segment_noisy["segment"], order_segment_noisy["segment"]+outlier_diff_rank, "--", c="#808080", alpha=0.15, lw=0.5, zorder=2)
    ax_rob_rank[3, clm].plot(order_segment_noisy["segment"], order_segment_noisy["segment"]-outlier_diff_rank, "--", c="#808080", alpha=0.15, lw=0.5, zorder=2)
    ax_rob_rank[4, clm].plot(order_segment_noisy["segment"], order_segment_noisy["segment"], "--", c="#808080", alpha=0.5, zorder=2)
    ax_rob_rank[4, clm].plot(order_segment_noisy["segment"], order_segment_noisy["segment"]+outlier_diff_rank, "--", c="#808080", alpha=0.15, lw=0.5, zorder=2)
    ax_rob_rank[4, clm].plot(order_segment_noisy["segment"], order_segment_noisy["segment"]-outlier_diff_rank, "--", c="#808080", alpha=0.15, lw=0.5, zorder=2)
    for i in range(1, noise_samples+1):
        ta_rank = order_segment_noisy[["dtu_ta_base", f"dtu_ta_{noisy}_{i}"]].dropna(axis=0, how='all').replace(to_replace=np.nan, value=np.inf)
        ax_rob_rank[0, clm].scatter(ta_rank["dtu_ta_base"], ta_rank[f"dtu_ta_{noisy}_{i}"], c=color_dtu_ta, s=3, alpha=0.1, zorder=3)
        dtu_ta_x += ta_rank["dtu_ta_base"].to_list()
        dtu_ta_y += ta_rank[f"dtu_ta_{noisy}_{i}"].to_list()
        ax_rob_rank[1, clm].scatter(order_segment_noisy["dtu_greedy_base"], order_segment_noisy[f"dtu_greedy_{noisy}_{i}"], c=color_dtu_greedy, s=3, alpha=0.1, zorder=3)
        dtu_greedy_x += order_segment_noisy["dtu_greedy_base"].to_list()
        dtu_greedy_y += order_segment_noisy[f"dtu_greedy_{noisy}_{i}"].to_list()
        ax_rob_rank[2, clm].scatter(order_segment_noisy["tud_tbhb_base"], order_segment_noisy[f"tud_tbhb_{noisy}_{i}"], c=color_tud_tbhb, s=3, alpha=0.1, zorder=3)
        tud_tbhb_x += order_segment_noisy["tud_tbhb_base"].to_list()
        tud_tbhb_y += order_segment_noisy[f"tud_tbhb_{noisy}_{i}"].to_list()
        ax_rob_rank[3, clm].scatter(order_segment_noisy["tud_tb_base"], order_segment_noisy[f"tud_tb_{noisy}_{i}"], c=color_tud_tb, s=3, alpha=0.1, zorder=3)
        tud_tb_x += order_segment_noisy["tud_tb_base"].to_list()
        tud_tb_y += order_segment_noisy[f"tud_tb_{noisy}_{i}"].to_list()
        ax_rob_rank[4, clm].scatter(order_segment_noisy["tud_p_base"], order_segment_noisy[f"tud_p_{noisy}_{i}"], c=color_tud_p, s=3, alpha=0.1, zorder=3)
        tud_p_x += order_segment_noisy["tud_p_base"].to_list()
        tud_p_y += order_segment_noisy[f"tud_p_{noisy}_{i}"].to_list()
        

    corr_coeff_dtu_ta, corr_pvalue_dtu_ta = stats.kendalltau(dtu_ta_x, dtu_ta_y)
    ax_rob_rank[0, clm].text(0.75, 0.05, fr"$\rho= {corr_coeff_dtu_ta:4.3f}$", fontsize=7, transform=ax_rob_rank[0, clm].transAxes)
    ax_rob_rank[0, clm].text(0.02, 0.95, chr(ord("a") + subplot_numbering), fontsize=7, fontweight="bold", transform=ax_rob_rank[0, clm].transAxes)
    ax_rob_rank[0, clm].set_aspect("equal")

    corr_coeff_dtu_greedy, corr_pvalue_dtu_greedy = stats.kendalltau(dtu_greedy_x, dtu_greedy_y)
    ax_rob_rank[1, clm].text(0.75, 0.05, fr"$\rho= {corr_coeff_dtu_greedy:4.3f}$", fontsize=7, transform=ax_rob_rank[1, clm].transAxes)
    ax_rob_rank[1, clm].text(0.02, 0.95, chr(ord("a") + subplot_numbering + 3), fontsize=7, fontweight="bold", transform=ax_rob_rank[1, clm].transAxes)
    ax_rob_rank[1, clm].set_aspect("equal")
    
    corr_coeff_tud_tbhb, corr_pvalue_tud_tud_tbhb = stats.kendalltau(tud_tbhb_x, tud_tbhb_y)
    ax_rob_rank[2, clm].text(0.75, 0.05, fr"$\rho = {corr_coeff_tud_tbhb:4.3f}$", fontsize=7, transform=ax_rob_rank[2, clm].transAxes)
    ax_rob_rank[2, clm].text(0.02, 0.95, chr(ord("a") + subplot_numbering + 6), fontsize=7, fontweight="bold", transform=ax_rob_rank[2, clm].transAxes)
    ax_rob_rank[2, clm].set_aspect("equal")
    
    corr_coeff_tud_tb, corr_pvalue_tud_tb = stats.kendalltau(tud_tb_x, tud_tb_y)
    ax_rob_rank[3, clm].text(0.75, 0.05, fr"$\rho= {corr_coeff_tud_tb:4.3f}$", fontsize=7, transform=ax_rob_rank[3, clm].transAxes)
    ax_rob_rank[3, clm].text(0.02, 0.95, chr(ord("a") + subplot_numbering + 9), fontsize=7, fontweight="bold", transform=ax_rob_rank[3, clm].transAxes)
    ax_rob_rank[3, clm].set_aspect("equal")
    
    corr_coeff_tud_p, corr_pvalue_tud_p = stats.kendalltau(tud_p_x, tud_p_y)
    ax_rob_rank[4, clm].text(0.75, 0.05, fr"$\rho= {corr_coeff_tud_p:4.3f}$", fontsize=7, transform=ax_rob_rank[4, clm].transAxes)
    ax_rob_rank[4, clm].text(0.02, 0.95, chr(ord("a") + subplot_numbering + 12), fontsize=7, fontweight="bold", transform=ax_rob_rank[4, clm].transAxes)
    ax_rob_rank[4, clm].tick_params(axis="x", labelsize=tick_fs, pad=1.5)
    ax_rob_rank[4, clm].set_aspect("equal")
    
    subplot_numbering += 1

ax_rob_rank[4, 0].set_xlim(0, 203)
ax_rob_rank[4, 0].xaxis.set_major_locator(ticker.MultipleLocator(50))
ax_rob_rank[4, 0].yaxis.set_major_locator(ticker.MultipleLocator(50))
ax_rob_rank[4, 0].set_ylim(0, 203)
ax_rob_rank[0, 0].tick_params(axis="y", labelsize=tick_fs, pad=1.5)
ax_rob_rank[1, 0].tick_params(axis="y", labelsize=tick_fs, pad=1.5)
ax_rob_rank[2, 0].tick_params(axis="y", labelsize=tick_fs, pad=1.5)
ax_rob_rank[3, 0].tick_params(axis="y", labelsize=tick_fs, pad=1.5)
ax_rob_rank[4, 0].tick_params(axis="y", labelsize=tick_fs, pad=1.5)

ax_rob_rank[4, 1].set_xlabel("Rank with Base Data", fontsize=axis_fs)
ax_rob_rank[0, 0].set_title("Noisy demand", fontsize=axis_fs)
ax_rob_rank[0, 1].set_title("Noisy costs", fontsize=axis_fs)
ax_rob_rank[0, 2].set_title("Noisy velocities", fontsize=axis_fs)

ax_rob_rank[0, 0].set_ylabel("Batched optimization \n Rank with Noisy Data", fontsize=axis_fs)
ax_rob_rank[1, 0].set_ylabel("Greedy optimization \n Rank with Noisy Data", fontsize=axis_fs)
ax_rob_rank[2, 0].set_ylabel(r"$Q_{\mathrm{dyn}}$"+"\nRank with Noisy Data", fontsize=axis_fs)
ax_rob_rank[3, 0].set_ylabel(r"$Q_{\mathrm{stat}}$"+"\nRank with Noisy Data", fontsize=axis_fs)
ax_rob_rank[4, 0].set_ylabel(r"$Q_{\mathrm{pen}}$"+"\nRank with Noisy Data", fontsize=axis_fs)

fig_rob_rank.savefig(join(figures, f"FIGS5_Robustness_Ranks.{file_format}"), bbox_inches="tight")
plt.show()

In [ ]:
fig_rob_npv_all, ax_rob_npv_all = plt.subplots(5, 3, figsize=(7.5, 9), dpi=dpi, sharey=True, sharex=True)
fig_rob_npv_all.tight_layout(pad=1.0, h_pad=0.5, w_pad=0.1)

subplot_numbering = 0

for clm, noisy in enumerate(["noisy_demand", "noisy_costs", "noisy_speeds"]):
    ax_rob_npv_all[0, clm].plot(monetary_data_noisy["t"][:last_year], 1000*(monetary_data_noisy[f"dtu_ta_{noisy}_npv"][:last_year]-monetary_data["dtu_ta_npv"][:last_year]), lw=1, c=color_dtu_ta)
    ax_rob_npv_all[1, clm].plot(monetary_data_noisy["t"][:last_year], 1000*(monetary_data_noisy[f"dtu_greedy_{noisy}_npv"][:last_year]-monetary_data["dtu_greedy_npv"][:last_year]), lw=1, c=color_dtu_greedy)
    ax_rob_npv_all[2, clm].plot(monetary_data_noisy["t"][:last_year], 1000*(monetary_data_noisy[f"tud_tbhb_{noisy}_npv"][:last_year]-monetary_data["tud_tbhb_npv"][:last_year]), lw=1, c=color_tud_tbhb)
    ax_rob_npv_all[3, clm].plot(monetary_data_noisy["t"][:last_year], 1000*(monetary_data_noisy[f"tud_tb_{noisy}_npv"][:last_year]-monetary_data["tud_tb_npv"][:last_year]), lw=1, c=color_tud_tb)
    ax_rob_npv_all[4, clm].plot(monetary_data_noisy["t"][:last_year], 1000*(monetary_data_noisy[f"tud_p_{noisy}_npv"][:last_year]-monetary_data["tud_p_npv"][:last_year]), lw=1, c=color_tud_p)

    for i in range(1, noise_samples+1):
        ax_rob_npv_all[0, clm].plot(monetary_data_noisy["t"][:last_year], 1000*(monetary_data_noisy[f"dtu_ta_{noisy}_{i}_npv"][:last_year]-monetary_data["dtu_ta_npv"][:last_year]), lw=1, c=color_dtu_ta, alpha=0.2)
        ax_rob_npv_all[1, clm].plot(monetary_data_noisy["t"][:last_year], 1000*(monetary_data_noisy[f"dtu_greedy_{noisy}_{i}_npv"][:last_year]-monetary_data["dtu_greedy_npv"][:last_year]), lw=1, c=color_dtu_greedy, alpha=0.2)
        ax_rob_npv_all[2, clm].plot(monetary_data_noisy["t"][:last_year], 1000*(monetary_data_noisy[f"tud_tbhb_{noisy}_{i}_npv"][:last_year]-monetary_data["tud_tbhb_npv"][:last_year]), lw=1, c=color_tud_tbhb, alpha=0.2)
        ax_rob_npv_all[3, clm].plot(monetary_data_noisy["t"][:last_year], 1000*(monetary_data_noisy[f"tud_tb_{noisy}_{i}_npv"][:last_year]-monetary_data["tud_tb_npv"][:last_year]), lw=1, c=color_tud_tb, alpha=0.2)
        ax_rob_npv_all[4, clm].plot(monetary_data_noisy["t"][:last_year], 1000*(monetary_data_noisy[f"tud_p_{noisy}_{i}_npv"][:last_year]-monetary_data["tud_p_npv"][:last_year]), lw=1, c=color_tud_p, alpha=0.2)

    ax_rob_npv_all[0, clm].text(0.02, 0.95, chr(ord("a") + subplot_numbering), fontsize=7, fontweight="bold", transform=ax_rob_npv_all[0, clm].transAxes)
    ax_rob_npv_all[1, clm].text(0.02, 0.95, chr(ord("a") + subplot_numbering + 3), fontsize=7, fontweight="bold", transform=ax_rob_npv_all[1, clm].transAxes)
    ax_rob_npv_all[2, clm].text(0.02, 0.95, chr(ord("a") + subplot_numbering + 6), fontsize=7, fontweight="bold", transform=ax_rob_npv_all[2, clm].transAxes)
    ax_rob_npv_all[3, clm].text(0.02, 0.95, chr(ord("a") + subplot_numbering + 9), fontsize=7, fontweight="bold", transform=ax_rob_npv_all[3, clm].transAxes)
    ax_rob_npv_all[4, clm].text(0.02, 0.95, chr(ord("a") + subplot_numbering + 12), fontsize=7, fontweight="bold", transform=ax_rob_npv_all[4, clm].transAxes)
    subplot_numbering += 1
    
    ax_rob_npv_all[4, clm].tick_params(axis="x", labelsize=tick_fs, pad=1.5)

ax_rob_npv_all[4, 0].set_xlim(0, last_year)
ax_rob_npv_all[4, 0].xaxis.set_major_locator(ticker.MultipleLocator(10))
ax_rob_npv_all[4, 0].set_ylim(-250, 250)
ax_rob_npv_all[4, 0].yaxis.set_major_locator(ticker.MultipleLocator(50))
ax_rob_npv_all[0, 0].tick_params(axis="y", labelsize=tick_fs, pad=1.5)
ax_rob_npv_all[1, 0].tick_params(axis="y", labelsize=tick_fs, pad=1.5)
ax_rob_npv_all[2, 0].tick_params(axis="y", labelsize=tick_fs, pad=1.5)
ax_rob_npv_all[3, 0].tick_params(axis="y", labelsize=tick_fs, pad=1.5)
ax_rob_npv_all[4, 0].tick_params(axis="y", labelsize=tick_fs, pad=1.5)

ax_rob_npv_all[4, 1].set_xlabel("Year", fontsize=axis_fs)
ax_rob_npv_all[0, 0].set_ylabel("Batched optimization \n Change in NPV [m EUR]", fontsize=axis_fs)
ax_rob_npv_all[1, 0].set_ylabel("Greedy optimization \n Change in NPV [m EUR]", fontsize=axis_fs)
ax_rob_npv_all[2, 0].set_ylabel(r"$Q_{\mathrm{dyn}}$"+"\n Change in NPV [m EUR]", fontsize=axis_fs)
ax_rob_npv_all[3, 0].set_ylabel(r"$Q_{\mathrm{stat}}$"+"\n Change in NPV [m EUR]", fontsize=axis_fs)
ax_rob_npv_all[4, 0].set_ylabel(r"$Q_{\mathrm{pen}}$"+"\n Change in NPV [m EUR]", fontsize=axis_fs)

ax_rob_npv_all[0, 0].set_title("Noisy demand", fontsize=axis_fs)
ax_rob_npv_all[0, 1].set_title("Noisy costs", fontsize=axis_fs)
ax_rob_npv_all[0, 2].set_title("Noisy velocities", fontsize=axis_fs)

fig_rob_npv_all.savefig(join(figures, f"FIGS6_Robustness_NPV_all.{file_format}"))
plt.show()

In [ ]:
fig_rob_tb_all, ax_rob_tb_all = plt.subplots(5, 3, figsize=(7.5, 9), dpi=dpi, sharey=True, sharex=True)
fig_rob_tb_all.tight_layout(pad=1.0, h_pad=0.5, w_pad=0.1)

subplot_numbering = 0

for clm, noisy in enumerate(["noisy_demand", "noisy_costs", "noisy_speeds"]):
    ax_rob_tb_all[0, clm].plot(monetary_data_noisy["t"][:last_year], 1000*(monetary_data_noisy[f"dtu_ta_{noisy}_tb"][:last_year]-monetary_data["dtu_ta_tb"][:last_year]), lw=1, c=color_dtu_ta)
    ax_rob_tb_all[1, clm].plot(monetary_data_noisy["t"][:last_year], 1000*(monetary_data_noisy[f"dtu_greedy_{noisy}_tb"][:last_year]-monetary_data["dtu_greedy_tb"][:last_year]), lw=1, c=color_dtu_greedy)
    ax_rob_tb_all[2, clm].plot(monetary_data_noisy["t"][:last_year], 1000*(monetary_data_noisy[f"tud_tbhb_{noisy}_tb"][:last_year]-monetary_data["tud_tbhb_tb"][:last_year]), lw=1, c=color_tud_tbhb)
    ax_rob_tb_all[3, clm].plot(monetary_data_noisy["t"][:last_year], 1000*(monetary_data_noisy[f"tud_tb_{noisy}_tb"][:last_year]-monetary_data["tud_tb_tb"][:last_year]), lw=1, c=color_tud_tb)
    ax_rob_tb_all[4, clm].plot(monetary_data_noisy["t"][:last_year], 1000*(monetary_data_noisy[f"tud_p_{noisy}_tb"][:last_year]-monetary_data["tud_p_tb"][:last_year]), lw=1, c=color_tud_p)

    for i in range(1, noise_samples+1):
        ax_rob_tb_all[0, clm].plot(monetary_data_noisy["t"][:last_year], 1000*(monetary_data_noisy[f"dtu_ta_{noisy}_{i}_tb"][:last_year]-monetary_data["dtu_ta_tb"][:last_year]), lw=1, c=color_dtu_ta, alpha=0.2)
        ax_rob_tb_all[1, clm].plot(monetary_data_noisy["t"][:last_year], 1000*(monetary_data_noisy[f"dtu_greedy_{noisy}_{i}_tb"][:last_year]-monetary_data["dtu_greedy_tb"][:last_year]), lw=1, c=color_dtu_greedy, alpha=0.2)
        ax_rob_tb_all[2, clm].plot(monetary_data_noisy["t"][:last_year], 1000*(monetary_data_noisy[f"tud_tbhb_{noisy}_{i}_tb"][:last_year]-monetary_data["tud_tbhb_tb"][:last_year]), lw=1, c=color_tud_tbhb, alpha=0.2)
        ax_rob_tb_all[3, clm].plot(monetary_data_noisy["t"][:last_year], 1000*(monetary_data_noisy[f"tud_tb_{noisy}_{i}_tb"][:last_year]-monetary_data["tud_tb_tb"][:last_year]), lw=1, c=color_tud_tb, alpha=0.2)
        ax_rob_tb_all[4, clm].plot(monetary_data_noisy["t"][:last_year], 1000*(monetary_data_noisy[f"tud_p_{noisy}_{i}_tb"][:last_year]-monetary_data["tud_p_tb"][:last_year]), lw=1, c=color_tud_p, alpha=0.2)

    ax_rob_tb_all[0, clm].text(0.02, 0.95, chr(ord("a") + subplot_numbering), fontsize=7, fontweight="bold", transform=ax_rob_tb_all[0, clm].transAxes)
    ax_rob_tb_all[1, clm].text(0.02, 0.95, chr(ord("a") + subplot_numbering + 3), fontsize=7, fontweight="bold", transform=ax_rob_tb_all[1, clm].transAxes)
    ax_rob_tb_all[2, clm].text(0.02, 0.95, chr(ord("a") + subplot_numbering + 6), fontsize=7, fontweight="bold", transform=ax_rob_tb_all[2, clm].transAxes)
    ax_rob_tb_all[3, clm].text(0.02, 0.95, chr(ord("a") + subplot_numbering + 9), fontsize=7, fontweight="bold", transform=ax_rob_tb_all[3, clm].transAxes)
    ax_rob_tb_all[4, clm].text(0.02, 0.95, chr(ord("a") + subplot_numbering + 12), fontsize=7, fontweight="bold", transform=ax_rob_tb_all[4, clm].transAxes)
    subplot_numbering += 1
    
    ax_rob_tb_all[4, clm].tick_params(axis="x", labelsize=tick_fs, pad=1.5)

ax_rob_tb_all[4, 0].set_xlim(0, last_year)
ax_rob_tb_all[4, 0].xaxis.set_major_locator(ticker.MultipleLocator(10))
ax_rob_tb_all[4, 0].set_ylim(-40, 40)
ax_rob_tb_all[4, 0].yaxis.set_major_locator(ticker.MultipleLocator(10))
ax_rob_tb_all[4, 0].tick_params(axis="x", labelsize=tick_fs, pad=1.5)
ax_rob_tb_all[0, 0].tick_params(axis="y", labelsize=tick_fs, pad=1.5)
ax_rob_tb_all[1, 0].tick_params(axis="y", labelsize=tick_fs, pad=1.5)
ax_rob_tb_all[2, 0].tick_params(axis="y", labelsize=tick_fs, pad=1.5)
ax_rob_tb_all[3, 0].tick_params(axis="y", labelsize=tick_fs, pad=1.5)
ax_rob_tb_all[4, 0].tick_params(axis="y", labelsize=tick_fs, pad=1.5)

ax_rob_tb_all[4, 1].set_xlabel("Year", fontsize=axis_fs)
ax_rob_tb_all[0, 0].set_ylabel("Batched optimization \n Change in TB [m EUR]", fontsize=axis_fs)
ax_rob_tb_all[1, 0].set_ylabel("Greedy optimization \n Change in TB [m EUR]", fontsize=axis_fs)
ax_rob_tb_all[2, 0].set_ylabel(r"$Q_{\mathrm{dyn}}$"+"\nChange in TB [m EUR]", fontsize=axis_fs)
ax_rob_tb_all[3, 0].set_ylabel(r"$Q_{\mathrm{stat}}$"+"\nChange in TB [m EUR]", fontsize=axis_fs)
ax_rob_tb_all[4, 0].set_ylabel(r"$Q_{\mathrm{pen}}$"+"\nChange in TB [m EUR]", fontsize=axis_fs)

ax_rob_tb_all[0, 0].set_title("Noisy demand", fontsize=axis_fs)
ax_rob_tb_all[0, 1].set_title("Noisy costs", fontsize=axis_fs)
ax_rob_tb_all[0, 2].set_title("Noisy velocities", fontsize=axis_fs)

fig_rob_tb_all.savefig(join(figures, f"FIGS7_Robustness_TB_all.{file_format}"))
plt.show()

In [ ]:
fig_rob_hb_all, ax_rob_hb_all = plt.subplots(5, 3, figsize=(7.5, 9), dpi=dpi, sharey=True, sharex=True)
fig_rob_hb_all.tight_layout(pad=1.0, h_pad=0.5, w_pad=0.1)

subplot_numbering = 0

for clm, noisy in enumerate(["noisy_demand", "noisy_costs", "noisy_speeds"]):
    ax_rob_hb_all[0, clm].plot(monetary_data_noisy["t"][:last_year], 1000*(monetary_data_noisy[f"dtu_ta_{noisy}_hb"][:last_year]-monetary_data["dtu_ta_hb"][:last_year]), lw=1, c=color_dtu_ta)
    ax_rob_hb_all[1, clm].plot(monetary_data_noisy["t"][:last_year], 1000*(monetary_data_noisy[f"dtu_greedy_{noisy}_hb"][:last_year]-monetary_data["dtu_greedy_hb"][:last_year]), lw=1, c=color_dtu_greedy)
    ax_rob_hb_all[2, clm].plot(monetary_data_noisy["t"][:last_year], 1000*(monetary_data_noisy[f"tud_tbhb_{noisy}_hb"][:last_year]-monetary_data["tud_tbhb_hb"][:last_year]), lw=1, c=color_tud_tbhb)
    ax_rob_hb_all[3, clm].plot(monetary_data_noisy["t"][:last_year], 1000*(monetary_data_noisy[f"tud_tb_{noisy}_hb"][:last_year]-monetary_data["tud_tb_hb"][:last_year]), lw=1, c=color_tud_tb)
    ax_rob_hb_all[4, clm].plot(monetary_data_noisy["t"][:last_year], 1000*(monetary_data_noisy[f"tud_p_{noisy}_hb"][:last_year]-monetary_data["tud_p_hb"][:last_year]), lw=1, c=color_tud_p)

    for i in range(1, noise_samples+1):
        ax_rob_hb_all[0, clm].plot(monetary_data_noisy["t"][:last_year], 1000*(monetary_data_noisy[f"dtu_ta_{noisy}_{i}_hb"][:last_year]-monetary_data["dtu_ta_hb"][:last_year]), lw=1, c=color_dtu_ta, alpha=0.2)
        ax_rob_hb_all[1, clm].plot(monetary_data_noisy["t"][:last_year], 1000*(monetary_data_noisy[f"dtu_greedy_{noisy}_{i}_hb"][:last_year]-monetary_data["dtu_greedy_hb"][:last_year]), lw=1, c=color_dtu_greedy, alpha=0.2)
        ax_rob_hb_all[2, clm].plot(monetary_data_noisy["t"][:last_year], 1000*(monetary_data_noisy[f"tud_tbhb_{noisy}_{i}_hb"][:last_year]-monetary_data["tud_tbhb_hb"][:last_year]), lw=1, c=color_tud_tbhb, alpha=0.2)
        ax_rob_hb_all[3, clm].plot(monetary_data_noisy["t"][:last_year], 1000*(monetary_data_noisy[f"tud_tb_{noisy}_{i}_hb"][:last_year]-monetary_data["tud_tb_hb"][:last_year]), lw=1, c=color_tud_tb, alpha=0.2)
        ax_rob_hb_all[4, clm].plot(monetary_data_noisy["t"][:last_year], 1000*(monetary_data_noisy[f"tud_p_{noisy}_{i}_hb"][:last_year]-monetary_data["tud_p_hb"][:last_year]), lw=1, c=color_tud_p, alpha=0.2)

    ax_rob_hb_all[0, clm].text(0.02, 0.95, chr(ord("a") + subplot_numbering), fontsize=7, fontweight="bold", transform=ax_rob_hb_all[0, clm].transAxes)
    ax_rob_hb_all[1, clm].text(0.02, 0.95, chr(ord("a") + subplot_numbering + 3), fontsize=7, fontweight="bold", transform=ax_rob_hb_all[1, clm].transAxes)
    ax_rob_hb_all[2, clm].text(0.02, 0.95, chr(ord("a") + subplot_numbering + 6), fontsize=7, fontweight="bold", transform=ax_rob_hb_all[2, clm].transAxes)
    ax_rob_hb_all[3, clm].text(0.02, 0.95, chr(ord("a") + subplot_numbering + 9), fontsize=7, fontweight="bold", transform=ax_rob_hb_all[3, clm].transAxes)
    ax_rob_hb_all[4, clm].text(0.02, 0.95, chr(ord("a") + subplot_numbering + 12), fontsize=7, fontweight="bold", transform=ax_rob_hb_all[4, clm].transAxes)
    subplot_numbering += 1
    
    ax_rob_hb_all[4, clm].tick_params(axis="x", labelsize=tick_fs, pad=1.5)

ax_rob_hb_all[4, 0].set_xlim(0, last_year)
ax_rob_hb_all[4, 0].xaxis.set_major_locator(ticker.MultipleLocator(10))
ax_rob_hb_all[4, 0].set_ylim(-250, 100)
ax_rob_hb_all[4, 0].yaxis.set_major_locator(ticker.MultipleLocator(50))
ax_rob_hb_all[4, 0].tick_params(axis="x", labelsize=tick_fs, pad=1.5)
ax_rob_hb_all[0, 0].tick_params(axis="y", labelsize=tick_fs, pad=1.5)
ax_rob_hb_all[1, 0].tick_params(axis="y", labelsize=tick_fs, pad=1.5)
ax_rob_hb_all[2, 0].tick_params(axis="y", labelsize=tick_fs, pad=1.5)
ax_rob_hb_all[3, 0].tick_params(axis="y", labelsize=tick_fs, pad=1.5)
ax_rob_hb_all[4, 0].tick_params(axis="y", labelsize=tick_fs, pad=1.5)

ax_rob_hb_all[4, 1].set_xlabel("Year", fontsize=axis_fs)
ax_rob_hb_all[0, 0].set_ylabel("Batched optimization \n Change in HB [m EUR]", fontsize=axis_fs)
ax_rob_hb_all[1, 0].set_ylabel("Greedy optimization \n Change in HB [m EUR]", fontsize=axis_fs)
ax_rob_hb_all[2, 0].set_ylabel(r"$Q_{\mathrm{dyn}}$"+"\nChange in HB [m EUR]", fontsize=axis_fs)
ax_rob_hb_all[3, 0].set_ylabel(r"$Q_{\mathrm{stat}}$"+"\nChange in HB [m EUR]", fontsize=axis_fs)
ax_rob_hb_all[4, 0].set_ylabel(r"$Q_{\mathrm{pen}}$"+"\nChange in HB [m EUR]", fontsize=axis_fs)

ax_rob_hb_all[0, 0].set_title("Noisy demand", fontsize=axis_fs)
ax_rob_hb_all[0, 1].set_title("Noisy costs", fontsize=axis_fs)
ax_rob_hb_all[0, 2].set_title("Noisy velocities", fontsize=axis_fs)

fig_rob_hb_all.savefig(join(figures, f"FIGS8_Robustness_HB_all.{file_format}"))
plt.show()

In [ ]:
fig_rob_ba_yearly_all, ax_rob_ba_yearly_all = plt.subplots(5, 3, figsize=(7.5, 9), dpi=dpi, sharey=True, sharex=True)
fig_rob_ba_yearly_all.tight_layout(pad=1.0, h_pad=0.5, w_pad=0.1)

subplot_numbering = 0

for clm, noisy in enumerate(["noisy_demand", "noisy_costs", "noisy_speeds"]):
    ax_rob_ba_yearly_all[0, clm].plot(ba_noisy_year["t"][:last_year+1], ba_noisy_year[f"dtu_ta_{noisy}"][:last_year+1]-ba_year["dtu_ta"][:last_year+1], lw=1, c=color_dtu_ta)
    ax_rob_ba_yearly_all[1, clm].plot(ba_noisy_year["t"][:last_year+1], ba_noisy_year[f"dtu_greedy_{noisy}"][:last_year+1]-ba_year["dtu_greedy"][:last_year+1], lw=1, c=color_dtu_greedy)
    ax_rob_ba_yearly_all[2, clm].plot(ba_noisy_year["t"][:last_year+1], ba_noisy_year[f"tud_tbhb_{noisy}"][:last_year+1]-ba_year["tud_tbhb"][:last_year+1], lw=1, c=color_tud_tbhb)
    ax_rob_ba_yearly_all[3, clm].plot(ba_noisy_year["t"][:last_year+1], ba_noisy_year[f"tud_tb_{noisy}"][:last_year+1]-ba_year["tud_tb"][:last_year+1], lw=1, c=color_tud_tb)
    ax_rob_ba_yearly_all[4, clm].plot(ba_noisy_year["t"][:last_year+1], ba_noisy_year[f"tud_p_{noisy}"][:last_year+1]-ba_year["tud_p"][:last_year+1], lw=1, c=color_tud_p)

    for i in range(1, noise_samples+1):
        ax_rob_ba_yearly_all[0, clm].plot(ba_noisy_year["t"][:last_year+1], ba_noisy_year[f"dtu_ta_{noisy}_{i}"][:last_year+1]-ba_year["dtu_ta"][:last_year+1], lw=1, c=color_dtu_ta, alpha=0.2)
        ax_rob_ba_yearly_all[1, clm].plot(ba_noisy_year["t"][:last_year+1], ba_noisy_year[f"dtu_greedy_{noisy}_{i}"][:last_year+1]-ba_year["dtu_greedy"][:last_year+1], lw=1, c=color_dtu_greedy, alpha=0.2)
        ax_rob_ba_yearly_all[2, clm].plot(ba_noisy_year["t"][:last_year+1], ba_noisy_year[f"tud_tbhb_{noisy}_{i}"][:last_year+1]-ba_year["tud_tbhb"][:last_year+1], lw=1, c=color_tud_tbhb, alpha=0.2)
        ax_rob_ba_yearly_all[3, clm].plot(ba_noisy_year["t"][:last_year+1], ba_noisy_year[f"tud_tb_{noisy}_{i}"][:last_year+1]-ba_year["tud_tb"][:last_year+1], lw=1, c=color_tud_tb, alpha=0.2)
        ax_rob_ba_yearly_all[4, clm].plot(ba_noisy_year["t"][:last_year+1], ba_noisy_year[f"tud_p_{noisy}_{i}"][:last_year+1]-ba_year["tud_p"][:last_year+1], lw=1, c=color_tud_p, alpha=0.2)

    ax_rob_ba_yearly_all[0, clm].text(0.02, 0.95, chr(ord("a") + subplot_numbering), fontsize=7, fontweight="bold", transform=ax_rob_ba_yearly_all[0, clm].transAxes)
    ax_rob_ba_yearly_all[1, clm].text(0.02, 0.95, chr(ord("a") + subplot_numbering + 3), fontsize=7, fontweight="bold", transform=ax_rob_ba_yearly_all[1, clm].transAxes)
    ax_rob_ba_yearly_all[2, clm].text(0.02, 0.95, chr(ord("a") + subplot_numbering + 6), fontsize=7, fontweight="bold", transform=ax_rob_ba_yearly_all[2, clm].transAxes)
    ax_rob_ba_yearly_all[3, clm].text(0.02, 0.95, chr(ord("a") + subplot_numbering + 9), fontsize=7, fontweight="bold", transform=ax_rob_ba_yearly_all[3, clm].transAxes)
    ax_rob_ba_yearly_all[4, clm].text(0.02, 0.95, chr(ord("a") + subplot_numbering + 12), fontsize=7, fontweight="bold", transform=ax_rob_ba_yearly_all[4, clm].transAxes)
    subplot_numbering += 1
    
    ax_rob_ba_yearly_all[4, clm].tick_params(axis="x", labelsize=tick_fs, pad=1.5)

ax_rob_ba_yearly_all[4, 0].set_xlim(0, last_year)
ax_rob_ba_yearly_all[4, 0].xaxis.set_major_locator(ticker.MultipleLocator(10))
ax_rob_ba_yearly_all[4, 0].set_ylim(-0.15, 0.15)
ax_rob_ba_yearly_all[4, 0].yaxis.set_major_locator(ticker.MultipleLocator(0.05))
ax_rob_ba_yearly_all[0, 0].tick_params(axis="y", labelsize=tick_fs, pad=1.5)
ax_rob_ba_yearly_all[1, 0].tick_params(axis="y", labelsize=tick_fs, pad=1.5)
ax_rob_ba_yearly_all[2, 0].tick_params(axis="y", labelsize=tick_fs, pad=1.5)
ax_rob_ba_yearly_all[3, 0].tick_params(axis="y", labelsize=tick_fs, pad=1.5)
ax_rob_ba_yearly_all[4, 0].tick_params(axis="y", labelsize=tick_fs, pad=1.5)

ax_rob_ba_yearly_all[4, 1].set_xlabel("Year", fontsize=axis_fs)
ax_rob_ba_yearly_all[0, 0].set_ylabel("Batched optimization \nChange in bikeability", fontsize=axis_fs)
ax_rob_ba_yearly_all[1, 0].set_ylabel("Greedy optimization \nChange in bikeability", fontsize=axis_fs)
ax_rob_ba_yearly_all[2, 0].set_ylabel(r"$Q_{\mathrm{dyn}}$"+"\nChange in bikeability", fontsize=axis_fs)
ax_rob_ba_yearly_all[3, 0].set_ylabel(r"$Q_{\mathrm{stat}}$"+"\nChange in bikeability", fontsize=axis_fs)
ax_rob_ba_yearly_all[4, 0].set_ylabel(r"$Q_{\mathrm{pen}}$"+"\nChange in bikeability", fontsize=axis_fs)

ax_rob_ba_yearly_all[0, 0].set_title("Noisy demand", fontsize=axis_fs)
ax_rob_ba_yearly_all[0, 1].set_title("Noisy costs", fontsize=axis_fs)
ax_rob_ba_yearly_all[0, 2].set_title("Noisy velocities", fontsize=axis_fs)

fig_rob_ba_yearly_all.savefig(join(figures, f"FIGS9_Robustness_BaY_all.{file_format}"))
plt.show()

In [ ]:
fig_rob_ba_segment_all, ax_rob_ba_segment_all = plt.subplots(5, 3, figsize=(7.5, 9), dpi=dpi, sharey=True, sharex=True)
fig_rob_ba_segment_all.tight_layout(pad=1.0, h_pad=0.5, w_pad=0.1)

subplot_numbering = 0

for clm, noisy in enumerate(["noisy_demand", "noisy_costs", "noisy_speeds"]):
    ax_rob_ba_segment_all[0, clm].plot(segments_added, ba_noisy_segment[f"dtu_ta_{noisy}"]-ba_segment["dtu_ta"], lw=1, c=color_dtu_ta)
    ax_rob_ba_segment_all[1, clm].plot(segments_added, ba_noisy_segment[f"dtu_greedy_{noisy}"]-ba_segment["dtu_greedy"], lw=1, c=color_dtu_greedy)
    ax_rob_ba_segment_all[2, clm].plot(segments_added, ba_noisy_segment[f"tud_tbhb_{noisy}"]-ba_segment["tud_tbhb"], lw=1, c=color_tud_tbhb)
    ax_rob_ba_segment_all[3, clm].plot(segments_added, ba_noisy_segment[f"tud_tb_{noisy}"]-ba_segment["tud_tb"], lw=1, c=color_tud_tb)
    ax_rob_ba_segment_all[4, clm].plot(segments_added, ba_noisy_segment[f"tud_p_{noisy}"]-ba_segment["tud_p"], lw=1, c=color_tud_p)

    for i in range(1, noise_samples+1):
        ax_rob_ba_segment_all[0, clm].plot(segments_added, ba_noisy_segment[f"dtu_ta_{noisy}_{i}"]-ba_segment["dtu_ta"], lw=1, c=color_dtu_ta, alpha=0.2)
        ax_rob_ba_segment_all[1, clm].plot(segments_added, ba_noisy_segment[f"dtu_greedy_{noisy}_{i}"]-ba_segment["dtu_greedy"], lw=1, c=color_dtu_greedy, alpha=0.2)
        ax_rob_ba_segment_all[2, clm].plot(segments_added, ba_noisy_segment[f"tud_tbhb_{noisy}_{i}"]-ba_segment["tud_tbhb"], lw=1, c=color_tud_tbhb, alpha=0.2)
        ax_rob_ba_segment_all[3, clm].plot(segments_added, ba_noisy_segment[f"tud_tb_{noisy}_{i}"]-ba_segment["tud_tb"], lw=1, c=color_tud_tb, alpha=0.2)
        ax_rob_ba_segment_all[4, clm].plot(segments_added, ba_noisy_segment[f"tud_p_{noisy}_{i}"]-ba_segment["tud_p"], lw=1, c=color_tud_p, alpha=0.2)

    ax_rob_ba_segment_all[0, clm].text(0.02, 0.95, chr(ord("a") + subplot_numbering), fontsize=7, fontweight="bold", transform=ax_rob_ba_segment_all[0, clm].transAxes)
    ax_rob_ba_segment_all[1, clm].text(0.02, 0.95, chr(ord("a") + subplot_numbering + 3), fontsize=7, fontweight="bold", transform=ax_rob_ba_segment_all[1, clm].transAxes)
    ax_rob_ba_segment_all[2, clm].text(0.02, 0.95, chr(ord("a") + subplot_numbering + 6), fontsize=7, fontweight="bold", transform=ax_rob_ba_segment_all[2, clm].transAxes)
    ax_rob_ba_segment_all[3, clm].text(0.02, 0.95, chr(ord("a") + subplot_numbering + 9), fontsize=7, fontweight="bold", transform=ax_rob_ba_segment_all[3, clm].transAxes)
    ax_rob_ba_segment_all[4, clm].text(0.02, 0.95, chr(ord("a") + subplot_numbering + 12), fontsize=7, fontweight="bold", transform=ax_rob_ba_segment_all[4, clm].transAxes)
    subplot_numbering += 1
    
    ax_rob_ba_segment_all[3, clm].tick_params(axis="x", labelsize=tick_fs, pad=1.5)

ax_rob_ba_segment_all[4, 0].set_xlim(0, max(segments_added))
ax_rob_ba_segment_all[4, 0].xaxis.set_major_locator(ticker.MultipleLocator(50))
ax_rob_ba_segment_all[4, 0].set_ylim(-0.15, 0.15)
ax_rob_ba_segment_all[4, 0].yaxis.set_major_locator(ticker.MultipleLocator(0.05))
ax_rob_ba_segment_all[0, 0].tick_params(axis="y", labelsize=tick_fs, pad=1.5)
ax_rob_ba_segment_all[1, 0].tick_params(axis="y", labelsize=tick_fs, pad=1.5)
ax_rob_ba_segment_all[2, 0].tick_params(axis="y", labelsize=tick_fs, pad=1.5)
ax_rob_ba_segment_all[3, 0].tick_params(axis="y", labelsize=tick_fs, pad=1.5)
ax_rob_ba_segment_all[4, 0].tick_params(axis="y", labelsize=tick_fs, pad=1.5)

ax_rob_ba_segment_all[4, 1].set_xlabel("Number of segments added", fontsize=axis_fs)
ax_rob_ba_segment_all[0, 0].set_ylabel("Batched optimization \nChange in bikeability", fontsize=axis_fs)
ax_rob_ba_segment_all[1, 0].set_ylabel("Greedy optimization \nChange in bikeability", fontsize=axis_fs)
ax_rob_ba_segment_all[2, 0].set_ylabel(r"$Q_{\mathrm{dyn}}$"+"\nChange in bikeability", fontsize=axis_fs)
ax_rob_ba_segment_all[3, 0].set_ylabel(r"$Q_{\mathrm{stat}}$"+"\nChange in bikeability", fontsize=axis_fs)
ax_rob_ba_segment_all[4, 0].set_ylabel(r"$Q_{\mathrm{pen}}$"+"\nChange in bikeability", fontsize=axis_fs)

ax_rob_ba_segment_all[0, 0].set_title("Noisy demand", fontsize=axis_fs)
ax_rob_ba_segment_all[0, 1].set_title("Noisy costs", fontsize=axis_fs)
ax_rob_ba_segment_all[0, 2].set_title("Noisy velocities", fontsize=axis_fs)

fig_rob_ba_segment_all.savefig(join(figures, f"FIGS10_Robustness_BaS_all.{file_format}"))
plt.show()